<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/01_oneformer_evaluador_mascaras_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
Sistema de Evaluación Avanzado OneFormer v1.0
==============================================================
Framework completo con máscaras separadas y logging corregido
Adaptado de Mask2Former para OneFormer (Universal Segmentation)

Características principales:
- Soporte para segmentación UNIVERSAL (instancia, semántica Y panóptica)
- Máscaras raw en archivos NPZ separados (comprimidos)
- JSON principal ligero con referencias
- Sistema de logging funcional
- Nombres sanitizados para GitHub
- Análisis geométrico con Shapely
- Características de textura con Mahotas y Scikit-image
- Exportación dual: JSON propio + formato COCO

Autor: Jesús L.
Proyecto: TFM - Evaluación Comparativa de Técnicas de Segmentación
Universidad: Universidad Oberta de Cataluña
Fecha: 2025
"""

'\nSistema de Evaluación Avanzado OneFormer v1.0\n==============================================================\nFramework completo con máscaras separadas y logging corregido\nAdaptado de Mask2Former para OneFormer (Universal Segmentation)\n\nCaracterísticas principales:\n- Soporte para segmentación UNIVERSAL (instancia, semántica Y panóptica)\n- Máscaras raw en archivos NPZ separados (comprimidos)\n- JSON principal ligero con referencias\n- Sistema de logging funcional\n- Nombres sanitizados para GitHub\n- Análisis geométrico con Shapely\n- Características de textura con Mahotas y Scikit-image\n- Exportación dual: JSON propio + formato COCO\n\nAutor: Jesús L.\nProyecto: TFM - Evaluación Comparativa de Técnicas de Segmentación\nUniversidad: Universidad Oberta de Cataluña\nFecha: 2025\n'

In [2]:
# =============================================================================
# INSTALACIÓN DE DEPENDENCIAS
# =============================================================================

"""
!pip install -q shapely>=1.8.0
!pip install -q mahotas>=1.4.0
!pip install -q scikit-image>=0.19.0
!pip install -q transformers
!pip install -q torch torchvision
"""

In [15]:
# =============================================================================
# IMPORTS
# =============================================================================

import torch
import numpy as np
import cv2
import json
import time
import os
import psutil
import logging
import hashlib
import warnings
from datetime import datetime
from pathlib import Path
from dataclasses import dataclass, asdict
from typing import Dict, List, Any, Optional, Tuple

# Librerías especializadas
import mahotas as mh
from shapely.geometry import Polygon, Point
from shapely.ops import unary_union
from shapely import affinity
from skimage import measure, morphology, feature, filters, segmentation
from skimage.color import rgb2gray, rgb2hsv, rgb2lab
from sklearn.cluster import KMeans

# IMPORTANTE: Cambiar imports para OneFormer
from transformers import OneFormerProcessor, OneFormerForUniversalSegmentation
from PIL import Image, ImageStat
import matplotlib.pyplot as plt
from tqdm import tqdm

# Configuración para entorno Colab
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    IN_COLAB = True
except:
    IN_COLAB = False
    print("No estamos en Colab, continuando sin montar Drive...")

# Suprimir warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# =============================================================================
# CONFIGURACIÓN Y ESTRUCTURAS DE DATOS
# =============================================================================

@dataclass
class ModeloInfo:
    """Información completa de un modelo OneFormer"""
    nombre_hf: str
    nombre_corto: str
    tipo: str  # 'universal' - OneFormer es universal
    dataset: str  # 'COCO', 'ADE20K', 'Cityscapes'
    arquitectura: str  # 'Swin-Large', 'Swin-Tiny', 'DiNAT-Large'
    descripcion: str
    tareas_soportadas: List[str]  # ['semantic', 'instance', 'panoptic']

    def obtener_nombre_sanitizado(self) -> str:
        """Obtiene nombre sin información sensible para GitHub"""
        return f"o{hash(self.nombre_hf) % 1000:03d}_{self.tipo[:4]}_{self.dataset[:4].lower()}"

@dataclass
class ConfiguracionUmbrales:
    """Configuración de umbrales con metadatos"""
    nombre: str
    valores: List[float]
    descripcion: str

    def obtener_nombre_sanitizado(self) -> str:
        """Obtiene nombre corto para archivos"""
        return f"cfg_{self.nombre[:4]}"

@dataclass
class ConfigEvaluacion:
    """Configuración centralizada del sistema de evaluación"""

    # Rutas base
    BASE_PATH: Path = Path("/content/drive/MyDrive/TFM/oneformer")
    DATASET_PATH: Path = BASE_PATH / "imagenes"

    # Parámetros de procesamiento
    MAX_SIZE_IMAGEN: int = 1024
    MAX_IMAGENES_LOTE: int = 50
    GUARDAR_VISUALIZACIONES: bool = True
    LIMPIAR_CACHE_CADA: int = 25
    GENERAR_FORMATO_COCO: bool = True

    # NUEVO: Opciones de sanitización
    USAR_NOMBRES_SANITIZADOS: bool = True  # Cambiar a True para GitHub

    # NUEVO: Tarea de segmentación por defecto para OneFormer
    TAREA_SEGMENTACION: str = "instance"  # 'semantic', 'instance', o 'panoptic'

    # Modelos disponibles
    MODELOS: List[ModeloInfo] = None

    # Configuraciones de umbrales
    UMBRALES: Dict[str, ConfiguracionUmbrales] = None

    def __post_init__(self):
        if self.MODELOS is None:
            self.MODELOS = [
                ModeloInfo(
                    nombre_hf="shi-labs/oneformer_coco_swin_large",
                    nombre_corto="oneformer-swin-large-coco",
                    tipo='universal',
                    dataset='COCO',
                    arquitectura='Swin-Large',
                    descripcion='OneFormer Universal - COCO dataset - Mayor precisión',
                    tareas_soportadas=['semantic', 'instance', 'panoptic']
                ),
                ModeloInfo(
                    nombre_hf="shi-labs/oneformer_ade20k_swin_large",
                    nombre_corto="oneformer-swin-large-ade20k",
                    tipo='universal',
                    dataset='ADE20K',
                    arquitectura='Swin-Large',
                    descripcion='OneFormer Universal - ADE20K dataset - Balance eficiencia/precisión',
                    tareas_soportadas=['semantic', 'instance', 'panoptic']
                ),
                ModeloInfo(
                    nombre_hf="shi-labs/oneformer_ade20k_swin_tiny",
                    nombre_corto="oneformer-swin-tiny-ade20k",
                    tipo='universal',
                    dataset='ADE20K',
                    arquitectura='Swin-Tiny',
                    descripcion='OneFormer Universal - ADE20K dataset - Más rápido',
                    tareas_soportadas=['semantic', 'instance', 'panoptic']
                ),
                ModeloInfo(
                    nombre_hf="shi-labs/oneformer_coco_dinat_large",
                    nombre_corto="oneformer-dinat-large-coco",
                    tipo='universal',
                    dataset='COCO',
                    arquitectura='DiNAT-Large',
                    descripcion='OneFormer Universal - COCO con DiNAT backbone',
                    tareas_soportadas=['semantic', 'instance', 'panoptic']
                )
            ]

        if self.UMBRALES is None:
            self.UMBRALES = {
                'ultra_sensible': ConfiguracionUmbrales(
                    nombre='ultra_sensible',
                    valores=[0.0001, 0.001, 0.01, 0.1],
                    descripcion='Detecta cambios mínimos - Máxima sensibilidad'
                ),
                'alta_sensibilidad': ConfiguracionUmbrales(
                    nombre='alta_sensibilidad',
                    valores=[0.001, 0.01, 0.05, 0.1, 0.3],
                    descripcion='Sensibilidad alta para detección temprana'
                ),
                'sensibilidad_media': ConfiguracionUmbrales(
                    nombre='sensibilidad_media',
                    valores=[0.01, 0.1, 0.3, 0.5],
                    descripcion='Balance entre precisión y recall'
                ),
                'baja_sensibilidad': ConfiguracionUmbrales(
                    nombre='baja_sensibilidad',
                    valores=[0.3, 0.5, 0.7],
                    descripcion='Solo detecciones muy confiables'
                )
            }

    def crear_directorio_ejecucion(self) -> Path:
        """Crea directorio único para cada ejecución"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        directorio_ejecucion = self.BASE_PATH / "resultados" / f"ejecucion_{timestamp}"

        subdirectorios = ["logs"]

        if self.GUARDAR_VISUALIZACIONES:
            subdirectorios.append("visualizaciones")

        if self.GENERAR_FORMATO_COCO:
            subdirectorios.append("formato_coco")

        for subdir in subdirectorios:
            (directorio_ejecucion / subdir).mkdir(parents=True, exist_ok=True)

        return directorio_ejecucion

    def obtener_info_modelo(self, idx: int) -> Optional[ModeloInfo]:
        """Obtiene información de un modelo por índice"""
        if 0 <= idx < len(self.MODELOS):
            return self.MODELOS[idx]
        return None

    def obtener_config_umbral(self, nombre: str) -> Optional[ConfiguracionUmbrales]:
        """Obtiene configuración de umbral por nombre"""
        return self.UMBRALES.get(nombre)

    def validar_configuracion(self) -> bool:
        """Valida que la configuración sea correcta"""
        try:
            if not self.DATASET_PATH.exists():
                print(f"⚠️ Dataset path no existe: {self.DATASET_PATH}")
                return False

            if not self.MODELOS or len(self.MODELOS) == 0:
                return False

            if not self.UMBRALES or len(self.UMBRALES) == 0:
                return False

            for config_umbral in self.UMBRALES.values():
                if not config_umbral.valores or any(u < 0 or u > 1 for u in config_umbral.valores):
                    return False

            # Validar tarea de segmentación
            if self.TAREA_SEGMENTACION not in ['semantic', 'instance', 'panoptic']:
                print(f"⚠️ Tarea inválida: {self.TAREA_SEGMENTACION}")
                return False

            return True

        except Exception as e:
            print(f"Error en validación: {e}")
            return False

    def obtener_resumen_configuracion(self) -> Dict[str, Any]:
        """Obtiene resumen de la configuración actual"""
        return {
            'total_modelos': len(self.MODELOS),
            'modelos_disponibles': [m.nombre_corto for m in self.MODELOS],
            'total_configuraciones_umbral': len(self.UMBRALES),
            'configuraciones_umbral': list(self.UMBRALES.keys()),
            'total_combinaciones': len(self.MODELOS) * len(self.UMBRALES),
            'dataset_path': str(self.DATASET_PATH),
            'visualizaciones_habilitadas': self.GUARDAR_VISUALIZACIONES,
            'formato_coco_habilitado': self.GENERAR_FORMATO_COCO,
            'max_imagenes_lote': self.MAX_IMAGENES_LOTE,
            'nombres_sanitizados': self.USAR_NOMBRES_SANITIZADOS,
            'tarea_segmentacion': self.TAREA_SEGMENTACION,
            'framework': 'OneFormer'
        }

In [17]:
# =============================================================================
# SISTEMA DE LOGGING
# =============================================================================

class LoggerManager:
    """Gestor centralizado de logging"""

    def __init__(self, directorio_logs: Path):
        self.directorio_logs = directorio_logs
        self.directorio_logs.mkdir(parents=True, exist_ok=True)
        self.loggers = {}
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )

    def crear_logger(self, nombre: str) -> logging.Logger:
        """Crea un logger específico con handler de archivo"""
        if nombre in self.loggers:
            return self.loggers[nombre]

        logger = logging.getLogger(f"oneformer_{nombre}")
        logger.setLevel(logging.INFO)
        logger.propagate = False

        # Limpiar handlers existentes
        logger.handlers.clear()

        archivo_log = f"{nombre}_{self.timestamp}.log"
        archivo_path = self.directorio_logs / archivo_log

        # Handler de archivo
        handler_archivo = logging.FileHandler(archivo_path, encoding='utf-8', mode='a')
        handler_archivo.setLevel(logging.INFO)

        formatter = logging.Formatter(
            '%(asctime)s - %(levelname)-8s - %(message)s',
            datefmt='%H:%M:%S'
        )
        handler_archivo.setFormatter(formatter)
        logger.addHandler(handler_archivo)

        self.loggers[nombre] = logger
        logger.info(f"Logger inicializado: {nombre}")

        return logger

    def log_error(self, componente: str, error: Exception, contexto: str = "") -> None:
        """Método específico para logging de errores"""
        logger = self.loggers.get(componente)
        if not logger:
            logger = self.crear_logger(componente)

        mensaje_completo = f"{contexto}: {str(error)}" if contexto else str(error)
        logger.error(mensaje_completo)

    def info(self, componente: str, mensaje: str) -> None:
        """Log de información general"""
        logger = self.loggers.get(componente)
        if not logger:
            logger = self.crear_logger(componente)
        logger.info(mensaje)

    def warning(self, componente: str, mensaje: str) -> None:
        """Log de advertencias"""
        logger = self.loggers.get(componente)
        if not logger:
            logger = self.crear_logger(componente)
        logger.warning(mensaje)

    def limpiar_logs_vacios(self) -> None:
        """Limpia archivos de log vacíos o muy pequeños"""
        try:
            for archivo_log in self.directorio_logs.glob("*.log"):
                if archivo_log.stat().st_size < 100:
                    archivo_log.unlink()
        except Exception as e:
            print(f"Error limpiando logs: {e}")

    def obtener_estadisticas_logs(self) -> Dict[str, Any]:
        """Obtiene estadísticas de los logs generados"""
        archivos_log = list(self.directorio_logs.glob("*.log"))

        estadisticas = {
            'total_archivos_log': len(archivos_log),
            'componentes_activos': len(self.loggers),
            'archivos_detalle': []
        }

        for archivo in archivos_log:
            try:
                tamaño_kb = archivo.stat().st_size / 1024
                estadisticas['archivos_detalle'].append({
                    'archivo': archivo.name,
                    'tamaño_kb': round(tamaño_kb, 2)
                })
            except Exception:
                continue

        return estadisticas

    def cerrar_loggers(self) -> None:
        """Cierra todos los handlers de los loggers"""
        for logger in self.loggers.values():
            for handler in logger.handlers[:]:
                handler.close()
                logger.removeHandler(handler)

In [18]:
# =============================================================================
# GESTOR DE MÁSCARAS SEPARADAS (IGUAL QUE ANTES)
# =============================================================================

class GestorMascaras:
    """Gestor para almacenar máscaras en archivos NPZ separados"""

    def __init__(self, directorio_base: Path, logger: logging.Logger = None):
        self.directorio_base = directorio_base
        self.directorio_mascaras = directorio_base / "mascaras_raw"
        self.directorio_mascaras.mkdir(exist_ok=True, parents=True)
        self.logger = logger

        if self.logger:
            self.logger.info(f"Gestor de máscaras inicializado en: {self.directorio_mascaras}")

    def guardar_mascaras_imagen(self, nombre_imagen: str, mascaras_data: Dict[str, Any],
                                modelo_nombre: str, config_umbral: str) -> Optional[str]:
        """
        Guarda máscaras de una imagen en archivo NPZ comprimido

        Returns:
            Ruta relativa al archivo de máscaras
        """
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
            nombre_base = Path(nombre_imagen).stem
            nombre_archivo = f"{modelo_nombre}_{config_umbral}_{nombre_base}_{timestamp}.npz"

            archivo_path = self.directorio_mascaras / nombre_archivo

            arrays_to_save = {}
            metadatos = {
                'nombre_imagen': nombre_imagen,
                'modelo': modelo_nombre,
                'config_umbral': config_umbral,
                'timestamp': timestamp
            }

            contador_mascaras = 0
            for umbral_key, umbral_data in mascaras_data.items():
                mascaras_raw = umbral_data.get('mascaras_raw')
                mascara_raw = umbral_data.get('mascara_raw')

                if mascaras_raw is not None:
                    for i, mask in enumerate(mascaras_raw):
                        if mask is not None:
                            key = f"{umbral_key}_mask_{i}"
                            arrays_to_save[key] = np.array(mask, dtype=np.float32)
                            contador_mascaras += 1
                    metadatos[f"{umbral_key}_num_mascaras"] = len(mascaras_raw)
                    metadatos[f"{umbral_key}_scores"] = umbral_data.get('confianza_scores', [])

                elif mascara_raw is not None:
                    key = f"{umbral_key}_mask"
                    arrays_to_save[key] = np.array(mascara_raw, dtype=np.float32)
                    contador_mascaras += 1
                    metadatos[f"{umbral_key}_num_mascaras"] = 1
                    metadatos[f"{umbral_key}_scores"] = umbral_data.get('confianza_scores', [1.0])

            if contador_mascaras > 0:
                np.savez_compressed(archivo_path, metadatos=metadatos, **arrays_to_save)

                if self.logger:
                    self.logger.info(f"Máscaras guardadas: {nombre_archivo} ({contador_mascaras} máscaras)")

                return str(archivo_path.relative_to(self.directorio_base))
            else:
                if self.logger:
                    self.logger.warning(f"No hay máscaras para guardar: {nombre_imagen}")
                return None

        except Exception as e:
            if self.logger:
                self.logger.error(f"Error guardando máscaras para {nombre_imagen}: {str(e)}")
            return None

    def cargar_mascaras_imagen(self, ruta_relativa: str) -> Optional[Dict[str, Any]]:
        """Carga máscaras desde archivo NPZ"""
        try:
            archivo_path = self.directorio_base / ruta_relativa

            if not archivo_path.exists():
                if self.logger:
                    self.logger.error(f"Archivo de máscaras no encontrado: {archivo_path}")
                return None

            data = np.load(archivo_path, allow_pickle=True)

            resultado = {
                'metadatos': data['metadatos'].item(),
                'mascaras_por_umbral': {}
            }

            for key in data.files:
                if key.startswith('umbral_') and key != 'metadatos':
                    parts = key.split('_')
                    if len(parts) >= 2:
                        umbral_key = f"{parts[0]}_{parts[1]}"

                        if umbral_key not in resultado['mascaras_por_umbral']:
                            resultado['mascaras_por_umbral'][umbral_key] = []

                        resultado['mascaras_por_umbral'][umbral_key].append(data[key])

            if self.logger:
                self.logger.info(f"Máscaras cargadas desde: {ruta_relativa}")

            return resultado

        except Exception as e:
            if self.logger:
                self.logger.error(f"Error cargando máscaras desde {ruta_relativa}: {str(e)}")
            return None

    def obtener_estadisticas_almacenamiento(self) -> Dict[str, Any]:
        """Obtiene estadísticas de los archivos de máscaras"""
        archivos_npz = list(self.directorio_mascaras.glob("*.npz"))

        if not archivos_npz:
            return {
                'total_archivos': 0,
                'tamaño_total_mb': 0,
                'tamaño_promedio_mb': 0
            }

        tamaños = [f.stat().st_size for f in archivos_npz]
        tamaño_total = sum(tamaños)

        estadisticas = {
            'total_archivos': len(archivos_npz),
            'tamaño_total_mb': round(tamaño_total / (1024**2), 2),
            'tamaño_promedio_mb': round(np.mean(tamaños) / (1024**2), 2),
            'tamaño_min_mb': round(min(tamaños) / (1024**2), 2),
            'tamaño_max_mb': round(max(tamaños) / (1024**2), 2)
        }

        if self.logger:
            self.logger.info(f"Estadísticas máscaras: {estadisticas['total_archivos']} archivos, {estadisticas['tamaño_total_mb']} MB total")

        return estadisticas

In [19]:
# =============================================================================
# ANALIZADOR DE MÁSCARAS CON SHAPELY
# =============================================================================

class MaskAnalyzer:
    """Analizador especializado de máscaras de segmentación usando Shapely"""

    def __init__(self):
        self.epsilon_factor = 0.02

    def analizar_mascaras_completo(self, masks: np.ndarray, scores: List[float],
                                  imagen_shape: Tuple[int, int]) -> Dict[str, Any]:
        """Análisis completo de todas las máscaras detectadas"""
        if len(masks) == 0:
            return self._resultado_vacio()

        resultado = {
            'metadatos': {
                'num_mascaras': len(masks),
                'imagen_shape': imagen_shape,
                'total_pixels': imagen_shape[0] * imagen_shape[1]
            },
            'mascaras_individuales': [],
            'analisis_conjunto': {},
            'estadisticas_globales': {}
        }

        polygons_validos = []

        # Análisis individual de cada máscara
        for i, (mask, score) in enumerate(zip(masks, scores)):
            analisis_individual = self._analizar_mascara_individual(
                mask, score, i, imagen_shape
            )
            resultado['mascaras_individuales'].append(analisis_individual)

            if analisis_individual['shapely_geometry']['polygon_valido']:
                polygons_validos.append(analisis_individual['shapely_geometry']['polygon'])

        # Análisis conjunto
        if polygons_validos:
            resultado['analisis_conjunto'] = self._analizar_conjunto_mascaras(
                polygons_validos, imagen_shape
            )

        # Estadísticas globales
        resultado['estadisticas_globales'] = self._calcular_estadisticas_globales(
            resultado['mascaras_individuales']
        )

        return resultado

    def _analizar_mascara_individual(self, mask: np.ndarray, score: float,
                                   indice: int, imagen_shape: Tuple[int, int]) -> Dict[str, Any]:
        """Análisis completo de una máscara individual"""
        mask_binary = (mask > 0.5).astype(np.uint8)

        resultado = {
            'indice': indice,
            'score_confianza': float(score),
            'caracteristicas_geometricas': {},
            'caracteristicas_forma': {},
            'caracteristicas_contextuales': {},
            'shapely_geometry': {}
        }

        try:
            resultado['caracteristicas_geometricas'] = self._extraer_geometricas_basicas(
                mask_binary, imagen_shape
            )
            resultado['shapely_geometry'] = self._analizar_con_shapely(mask_binary)
            resultado['caracteristicas_forma'] = self._extraer_forma_calidad(mask_binary)
            resultado['caracteristicas_contextuales'] = self._extraer_contextuales(
                mask_binary, imagen_shape
            )
        except Exception as e:
            resultado['error'] = str(e)

        return resultado

    def _extraer_geometricas_basicas(self, mask: np.ndarray,
                                   imagen_shape: Tuple[int, int]) -> Dict[str, float]:
        """Extrae características geométricas básicas usando scikit-image"""
        props = measure.regionprops(mask.astype(int))[0] if np.any(mask) else None

        if props is None:
            return self._geometricas_vacias()

        total_pixels = imagen_shape[0] * imagen_shape[1]

        return {
            'area_pixels': float(props.area),
            'area_percentage': float(props.area / total_pixels * 100),
            'perimeter': float(props.perimeter),
            'compactness': float(4 * np.pi * props.area / (props.perimeter ** 2)) if props.perimeter > 0 else 0.0,
            'aspect_ratio': float(props.major_axis_length / props.minor_axis_length) if props.minor_axis_length > 0 else 0.0,
            'orientation_angle': float(np.degrees(props.orientation)),
            'centroid_y': float(props.centroid[0]),
            'centroid_x': float(props.centroid[1]),
            'solidity': float(props.solidity),
            'extent': float(props.extent),
            'eccentricity': float(props.eccentricity),
            'major_axis_length': float(props.major_axis_length),
            'minor_axis_length': float(props.minor_axis_length)
        }

    def _analizar_con_shapely(self, mask: np.ndarray) -> Dict[str, Any]:
        """Análisis geométrico avanzado usando Shapely"""
        try:
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if not contours:
                return self._shapely_vacio()

            contour = max(contours, key=cv2.contourArea)
            epsilon = self.epsilon_factor * cv2.arcLength(contour, True)
            contour_simplified = cv2.approxPolyDP(contour, epsilon, True)

            if len(contour_simplified) < 3:
                return self._shapely_vacio()

            coords = [(point[0][0], point[0][1]) for point in contour_simplified]
            if coords[0] != coords[-1]:
                coords.append(coords[0])

            polygon = Polygon(coords)

            if not polygon.is_valid:
                polygon = polygon.buffer(0)

            if not polygon.is_valid:
                return self._shapely_vacio()

            convex_hull = polygon.convex_hull

            return {
                'polygon_valido': True,
                'polygon': polygon,
                'area_shapely': float(polygon.area),
                'perimeter_shapely': float(polygon.length),
                'convex_hull_area': float(convex_hull.area),
                'convexity_ratio': float(polygon.area / convex_hull.area),
                'is_simple': bool(polygon.is_simple),
                'num_vertices': len(coords) - 1,
                'bounds': polygon.bounds,
                'centroid_shapely': (float(polygon.centroid.x), float(polygon.centroid.y))
            }

        except Exception as e:
            return {
                'polygon_valido': False,
                'error_shapely': str(e),
                **self._shapely_vacio()
            }

    def _extraer_forma_calidad(self, mask: np.ndarray) -> Dict[str, float]:
        """Características de forma y calidad de la máscara"""
        try:
            labeled_mask = measure.label(mask)
            num_components = labeled_mask.max()

            edges = cv2.Canny(mask.astype(np.uint8) * 255, 50, 150)
            edge_pixels = np.sum(edges > 0)

            grad_x = cv2.Sobel(mask.astype(np.float32), cv2.CV_32F, 1, 0, ksize=3)
            grad_y = cv2.Sobel(mask.astype(np.float32), cv2.CV_32F, 0, 1, ksize=3)
            gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
            edge_smoothness = float(np.std(gradient_magnitude[gradient_magnitude > 0]))

            filled = morphology.binary_fill_holes(mask)
            holes_area = np.sum(filled) - np.sum(mask)

            if np.sum(mask) > 0:
                equivalent_diameter = np.sqrt(4 * np.sum(mask) / np.pi)
                ideal_perimeter = np.pi * equivalent_diameter
                actual_perimeter = float(measure.perimeter(mask))
                roughness = actual_perimeter / ideal_perimeter if ideal_perimeter > 0 else 0.0
            else:
                roughness = 0.0

            return {
                'num_components': int(num_components),
                'edge_pixels': int(edge_pixels),
                'edge_smoothness': edge_smoothness,
                'holes_area': int(holes_area),
                'holes_percentage': float(holes_area / np.sum(filled) * 100) if np.sum(filled) > 0 else 0.0,
                'roughness_factor': roughness,
                'is_single_component': bool(num_components == 1)
            }

        except Exception as e:
            return {
                'error_forma': str(e),
                'num_components': 0,
                'edge_pixels': 0,
                'edge_smoothness': 0.0,
                'holes_area': 0,
                'holes_percentage': 0.0,
                'roughness_factor': 0.0,
                'is_single_component': False
            }

    def _extraer_contextuales(self, mask: np.ndarray,
                            imagen_shape: Tuple[int, int]) -> Dict[str, Any]:
        """Características contextuales de posición y distribución"""
        h, w = imagen_shape
        props = measure.regionprops(mask.astype(int))[0] if np.any(mask) else None

        if props is None:
            return self._contextuales_vacios()

        cy, cx = props.centroid
        pos_y_rel = cy / h
        pos_x_rel = cx / w

        position_region = self._clasificar_posicion(pos_x_rel, pos_y_rel)

        dist_top = cy
        dist_bottom = h - cy
        dist_left = cx
        dist_right = w - cx

        center_x, center_y = w / 2, h / 2
        dist_to_center = np.sqrt((cx - center_x)**2 + (cy - center_y)**2)
        dist_to_center_normalized = dist_to_center / np.sqrt(center_x**2 + center_y**2)

        bbox = props.bbox
        bbox_area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
        bbox_coverage = float(bbox_area / (h * w))

        return {
            'centroid_relative': {'x': float(pos_x_rel), 'y': float(pos_y_rel)},
            'position_region': position_region,
            'distances_to_edges': {
                'top': float(dist_top),
                'bottom': float(dist_bottom),
                'left': float(dist_left),
                'right': float(dist_right)
            },
            'distance_to_center': float(dist_to_center),
            'distance_to_center_normalized': float(dist_to_center_normalized),
            'bbox_coverage_percentage': float(bbox_coverage * 100),
            'aspect_vs_image': float((bbox[3] - bbox[1]) / (bbox[2] - bbox[0])) if (bbox[2] - bbox[0]) > 0 else 0.0
        }

    def _analizar_conjunto_mascaras(self, polygons: List[Polygon],
                                  imagen_shape: Tuple[int, int]) -> Dict[str, Any]:
        """Análisis de relaciones entre múltiples máscaras"""
        if len(polygons) < 2:
            return {'num_mascaras': len(polygons), 'analisis_conjunto': 'insuficientes_mascaras'}

        union_polygon = unary_union(polygons)
        overlaps = []
        distances = []

        for i in range(len(polygons)):
            for j in range(i + 1, len(polygons)):
                poly1, poly2 = polygons[i], polygons[j]

                intersection = poly1.intersection(poly2)
                overlap_area = intersection.area if hasattr(intersection, 'area') else 0
                overlap_percentage = (overlap_area / min(poly1.area, poly2.area)) * 100
                overlaps.append(overlap_percentage)

                distance = poly1.distance(poly2)
                distances.append(distance)

        centroids = [poly.centroid for poly in polygons]
        centroid_distances = []

        for i in range(len(centroids)):
            for j in range(i + 1, len(centroids)):
                dist = centroids[i].distance(centroids[j])
                centroid_distances.append(dist)

        return {
            'num_mascaras': len(polygons),
            'area_total_union': float(union_polygon.area),
            'coverage_percentage': float(union_polygon.area / (imagen_shape[0] * imagen_shape[1]) * 100),
            'overlaps': {
                'mean_overlap_percentage': float(np.mean(overlaps)),
                'max_overlap_percentage': float(np.max(overlaps)),
                'num_overlapping_pairs': int(np.sum(np.array(overlaps) > 0))
            },
            'distances': {
                'mean_distance': float(np.mean(distances)),
                'min_distance': float(np.min(distances)),
                'max_distance': float(np.max(distances))
            },
            'spatial_distribution': {
                'mean_centroid_distance': float(np.mean(centroid_distances)),
                'centroid_spread': float(np.std(centroid_distances))
            }
        }

    def _calcular_estadisticas_globales(self, mascaras_individuales: List[Dict]) -> Dict[str, Any]:
        """Estadísticas globales del conjunto de máscaras"""
        if not mascaras_individuales:
            return {}

        areas = []
        compactness = []
        scores = []
        roughness = []

        for mask_data in mascaras_individuales:
            if 'error' not in mask_data:
                geom = mask_data.get('caracteristicas_geometricas', {})
                forma = mask_data.get('caracteristicas_forma', {})

                areas.append(geom.get('area_percentage', 0))
                compactness.append(geom.get('compactness', 0))
                scores.append(mask_data.get('score_confianza', 0))
                roughness.append(forma.get('roughness_factor', 0))

        def safe_stats(values):
            if not values:
                return {'mean': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0}
            return {
                'mean': float(np.mean(values)),
                'std': float(np.std(values)),
                'min': float(np.min(values)),
                'max': float(np.max(values))
            }

        return {
            'areas_estadisticas': safe_stats(areas),
            'compactness_estadisticas': safe_stats(compactness),
            'scores_estadisticas': safe_stats(scores),
            'roughness_estadisticas': safe_stats(roughness),
            'mascaras_exitosas': len([m for m in mascaras_individuales if 'error' not in m])
        }

    # Métodos auxiliares
    def _resultado_vacio(self):
        return {
            'metadatos': {'num_mascaras': 0},
            'mascaras_individuales': [],
            'analisis_conjunto': {},
            'estadisticas_globales': {}
        }

    def _geometricas_vacias(self):
        return {k: 0.0 for k in ['area_pixels', 'area_percentage', 'perimeter',
                                'compactness', 'aspect_ratio', 'orientation_angle',
                                'centroid_y', 'centroid_x', 'solidity', 'extent',
                                'eccentricity', 'major_axis_length', 'minor_axis_length']}

    def _shapely_vacio(self):
        return {
            'polygon_valido': False,
            'polygon': None,
            'area_shapely': 0.0,
            'perimeter_shapely': 0.0,
            'convex_hull_area': 0.0,
            'convexity_ratio': 0.0,
            'is_simple': False,
            'num_vertices': 0,
            'bounds': (0, 0, 0, 0),
            'centroid_shapely': (0.0, 0.0)
        }

    def _contextuales_vacios(self):
        return {
            'centroid_relative': {'x': 0.0, 'y': 0.0},
            'position_region': 'desconocida',
            'distances_to_edges': {'top': 0.0, 'bottom': 0.0, 'left': 0.0, 'right': 0.0},
            'distance_to_center': 0.0,
            'distance_to_center_normalized': 0.0,
            'bbox_coverage_percentage': 0.0,
            'aspect_vs_image': 0.0
        }

    def _clasificar_posicion(self, x_rel: float, y_rel: float) -> str:
        """Clasifica la posición de la máscara en la imagen"""
        if y_rel < 0.33:
            if x_rel < 0.33:
                return 'superior_izquierda'
            elif x_rel < 0.67:
                return 'superior_centro'
            else:
                return 'superior_derecha'
        elif y_rel < 0.67:
            if x_rel < 0.33:
                return 'centro_izquierda'
            elif x_rel < 0.67:
                return 'centro'
            else:
                return 'centro_derecha'
        else:
            if x_rel < 0.33:
                return 'inferior_izquierda'
            elif x_rel < 0.67:
                return 'inferior_centro'
            else:
                return 'inferior_derecha'

# =============================================================================
# EXTRACTOR DE CARACTERÍSTICAS AVANZADAS
# =============================================================================

class ExtractorCaracteristicasAvanzado:
    """Extractor de características usando librerías especializadas"""

    def __init__(self):
        self.logger = None

    def set_logger(self, logger: logging.Logger):
        """Asigna logger para el extractor"""
        self.logger = logger

    def analizar_imagen_completa(self, imagen: Image.Image, ruta: str) -> Dict[str, Any]:
        """Análisis completo usando librerías especializadas"""
        img_array = np.array(imagen)
        img_gray = rgb2gray(img_array)
        img_hsv = rgb2hsv(img_array)
        img_lab = rgb2lab(img_array)

        img_gray_uint8 = (img_gray * 255).astype(np.uint8)

        caracteristicas = {
            'metadatos_basicos': self._extraer_metadatos_basicos(imagen, ruta),
            'color_y_paleta': self._analizar_color_avanzado(img_array, img_hsv, img_lab),
            'texturas_mahotas': self._extraer_texturas_mahotas(img_gray_uint8),
            'texturas_skimage': self._extraer_texturas_skimage(img_gray),
            'caracteristicas_geometricas': self._analizar_geometria_avanzada(img_gray_uint8),
            'multiscale_features': self._extraer_multiscale_features(img_gray),
            'propiedades_regionales': self._analizar_propiedades_regionales(img_gray),
            'descriptores_locales': self._extraer_descriptores_locales(img_gray_uint8)
        }

        return caracteristicas

    def _extraer_metadatos_basicos(self, imagen: Image.Image, ruta: str) -> Dict:
        """Metadatos básicos de la imagen"""
        w, h = imagen.size
        return {
            'dimensiones': {'ancho': w, 'alto': h},
            'aspecto_ratio': round(w / h, 3),
            'megapixeles': round((w * h) / 1000000, 2),
            'orientacion': 'horizontal' if w > h else 'vertical' if h > w else 'cuadrada',
            'formato': ruta.split('.')[-1].lower() if '.' in ruta else 'desconocido'
        }

    def _analizar_color_avanzado(self, img_rgb: np.ndarray, img_hsv: np.ndarray, img_lab: np.ndarray) -> Dict:
        """Análisis de color usando sklearn para paleta dominante"""
        pixels = img_rgb.reshape(-1, 3)
        kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
        kmeans.fit(pixels)

        colores_dominantes = kmeans.cluster_centers_.astype(int).tolist()
        proporciones = np.bincount(kmeans.labels_) / len(kmeans.labels_)

        return {
            'paleta_dominante': colores_dominantes,
            'proporciones_colores': proporciones.tolist(),
            'estadisticas_rgb': {
                'media': np.mean(img_rgb, axis=(0,1)).tolist(),
                'std': np.std(img_rgb, axis=(0,1)).tolist(),
                'rango': {
                    'min': np.min(img_rgb, axis=(0,1)).tolist(),
                    'max': np.max(img_rgb, axis=(0,1)).tolist()
                }
            },
            'hsv_global': {
                'hue_medio': float(np.mean(img_hsv[:,:,0])),
                'saturacion_media': float(np.mean(img_hsv[:,:,1])),
                'valor_medio': float(np.mean(img_hsv[:,:,2]))
            },
            'lab_luminancia': {
                'L_medio': float(np.mean(img_lab[:,:,0])),
                'a_medio': float(np.mean(img_lab[:,:,1])),
                'b_medio': float(np.mean(img_lab[:,:,2]))
            }
        }

    def _extraer_texturas_mahotas(self, img_gray: np.ndarray) -> Dict:
        """Extracción de características de textura usando Mahotas"""
        try:
            haralick_features = mh.features.haralick(img_gray, return_mean=True)
            lbp = mh.features.lbp(img_gray, radius=1, points=8, ignore_zeros=False)

            try:
                zernike_features = mh.features.zernike_moments(img_gray, radius=21)
            except:
                zernike_features = np.zeros(25)

            otsu_threshold = mh.otsu(img_gray)
            pftas = mh.features.pftas(img_gray)

            return {
                'haralick_features': haralick_features.tolist(),
                'lbp_histogram': np.histogram(lbp, bins=50)[0].tolist(),
                'zernike_moments': zernike_features.tolist(),
                'otsu_threshold': float(otsu_threshold),
                'pftas': pftas.tolist()
            }

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error en Mahotas: {str(e)}")
            return {
                'error': f"Error en Mahotas: {str(e)}",
                'haralick_features': [],
                'lbp_histogram': [],
                'zernike_moments': [],
                'otsu_threshold': 0.0,
                'pftas': []
            }

    def _extraer_texturas_skimage(self, img_gray: np.ndarray) -> Dict:
        """Extracción de características de textura usando Scikit-image"""
        try:
            lbp_skimage = feature.local_binary_pattern(img_gray, P=8, R=1, method='uniform')

            img_scaled = (img_gray * 255).astype(np.uint8)
            glcm = feature.graycomatrix(img_scaled, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4],
                                      levels=256, symmetric=True, normed=True)

            contrast = feature.graycoprops(glcm, 'contrast').mean()
            dissimilarity = feature.graycoprops(glcm, 'dissimilarity').mean()
            homogeneity = feature.graycoprops(glcm, 'homogeneity').mean()
            energy = feature.graycoprops(glcm, 'energy').mean()
            correlation = feature.graycoprops(glcm, 'correlation').mean()

            return {
                'lbp_uniform_histogram': np.histogram(lbp_skimage, bins=10)[0].tolist(),
                'glcm_properties': {
                    'contrast': float(contrast),
                    'dissimilarity': float(dissimilarity),
                    'homogeneity': float(homogeneity),
                    'energy': float(energy),
                    'correlation': float(correlation)
                }
            }

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error en Scikit-image: {str(e)}")
            return {
                'error': f"Error en Scikit-image: {str(e)}",
                'lbp_uniform_histogram': [],
                'glcm_properties': {}
            }

    def _analizar_geometria_avanzada(self, img_gray: np.ndarray) -> Dict:
        """Análisis geométrico usando mahotas y scikit-image"""
        try:
            edges_canny = feature.canny(img_gray / 255.0)
            edges_sobel = mh.sobel(img_gray)

            corners = feature.corner_harris(img_gray / 255.0)
            corner_peaks = feature.corner_peaks(corners, min_distance=5)

            moments = measure.moments(img_gray)
            centroid = measure.centroid(img_gray)

            return {
                'bordes_canny': float(np.sum(edges_canny)),
                'bordes_sobel_intensidad': float(np.mean(edges_sobel)),
                'num_corners': len(corner_peaks),
                'centroide': [float(centroid[0]), float(centroid[1])],
                'momentos_hu': measure.moments_hu(moments).tolist()
            }

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error en análisis geométrico: {str(e)}")
            return {
                'error': f"Error en análisis geométrico: {str(e)}",
                'bordes_canny': 0.0,
                'bordes_sobel_intensidad': 0.0,
                'num_corners': 0,
                'centroide': [0.0, 0.0],
                'momentos_hu': []
            }

    def _extraer_multiscale_features(self, img_gray: np.ndarray) -> Dict:
        """Características multi-escala usando scikit-image"""
        try:
            features_multiscale = feature.multiscale_basic_features(
                img_gray,
                intensity=True,
                edges=True,
                texture=True,
                sigma_min=0.5,
                sigma_max=8
            )

            return {
                'num_features': features_multiscale.shape[-1],
                'feature_means': np.mean(features_multiscale, axis=(0,1)).tolist(),
                'feature_stds': np.std(features_multiscale, axis=(0,1)).tolist()
            }

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error en features multiscale: {str(e)}")
            return {
                'error': f"Error en features multiscale: {str(e)}",
                'num_features': 0,
                'feature_means': [],
                'feature_stds': []
            }

    def _analizar_propiedades_regionales(self, img_gray: np.ndarray) -> Dict:
        """Análisis de propiedades regionales usando segmentación"""
        try:
            segments = segmentation.slic(img_gray, n_segments=100, compactness=10, channel_axis=None)
            regions = measure.regionprops(segments, intensity_image=img_gray)

            if regions:
                areas = [r.area for r in regions]
                eccentricities = [r.eccentricity for r in regions]
                intensities = [r.mean_intensity for r in regions]

                return {
                    'num_regiones': len(regions),
                    'area_promedio': float(np.mean(areas)),
                    'excentricidad_promedio': float(np.mean(eccentricities)),
                    'intensidad_promedio_regiones': float(np.mean(intensities)),
                    'variabilidad_areas': float(np.std(areas)),
                    'variabilidad_intensidades': float(np.std(intensities))
                }
            else:
                return {'num_regiones': 0}

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error en análisis regional: {str(e)}")
            return {
                'error': f"Error en análisis regional: {str(e)}",
                'num_regiones': 0
            }

    def _extraer_descriptores_locales(self, img_gray: np.ndarray) -> Dict:
        """Descriptores locales usando OpenCV"""
        try:
            img_cv = img_gray.copy()

            orb = cv2.ORB_create(nfeatures=100)
            keypoints_orb = orb.detect(img_cv, None)

            fast = cv2.FastFeatureDetector_create()
            keypoints_fast = fast.detect(img_cv, None)

            return {
                'orb_keypoints': len(keypoints_orb),
                'fast_keypoints': len(keypoints_fast),
                'keypoint_density': (len(keypoints_orb) + len(keypoints_fast)) / (img_gray.shape[0] * img_gray.shape[1])
            }

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error en descriptores locales: {str(e)}")
            return {
                'error': f"Error en descriptores locales: {str(e)}",
                'orb_keypoints': 0,
                'fast_keypoints': 0,
                'keypoint_density': 0.0
            }

# =============================================================================
# GENERADOR DE VISUALIZACIONES
# =============================================================================

class GeneradorVisualizaciones:
    """Genera visualizaciones comparativas imagen original vs detecciones"""

    def __init__(self, directorio_salida: Path, logger: logging.Logger = None):
        self.directorio_salida = directorio_salida
        self.directorio_salida.mkdir(parents=True, exist_ok=True)
        self.logger = logger

    def generar_visualizacion_completa(self, imagen_original: Image.Image,
                                     nombre_archivo: str, resultados_deteccion: Dict,
                                     umbral_principal: float = None,
                                     modelo_nombre: str = "modelo") -> Optional[str]:
        """Genera visualización completa: original + máscaras + overlay"""
        try:
            # Seleccionar umbral para visualización
            detecciones_umbral = resultados_deteccion.get('detecciones_por_umbral', {})
            if not detecciones_umbral:
                return None

            if umbral_principal is None:
                umbral_key = list(detecciones_umbral.keys())[0]
            else:
                umbral_key = f'umbral_{umbral_principal}'

            if umbral_key not in detecciones_umbral:
                return None

            datos_umbral = detecciones_umbral[umbral_key]

            # Crear figura con subplots
            fig, axes = plt.subplots(1, 3, figsize=(18, 6))
            fig.suptitle(f'Detección de Personas - {nombre_archivo}', fontsize=16, fontweight='bold')

            # 1. Imagen original
            axes[0].imshow(imagen_original)
            axes[0].set_title('Imagen Original', fontsize=14)
            axes[0].axis('off')

            # 2. Máscaras detectadas
            self._generar_visualizacion_mascaras(axes[1], imagen_original, datos_umbral, umbral_key)

            # 3. Overlay
            self._generar_overlay(axes[2], imagen_original, datos_umbral, umbral_key)

            # Información adicional
            personas = datos_umbral.get('personas_detectadas', 0)
            total = datos_umbral.get('total_detecciones', 0)
            umbral_val = umbral_key.replace('umbral_', '')

            fig.text(0.5, 0.02, f'Personas detectadas: {personas} | Total detecciones: {total} | Umbral: {umbral_val}',
                    ha='center', fontsize=12, style='italic')

            # Guardar visualización
            umbral_str = f"umbral_{umbral_principal:.4f}".replace('.', '')
            nombre_vis = f"vis_{modelo_nombre}_{umbral_str}_{nombre_archivo.replace('.jpg', '').replace('.png', '').replace('.jpeg', '')}.png"
            archivo_salida = self.directorio_salida / nombre_vis

            plt.tight_layout()
            plt.savefig(archivo_salida, dpi=150, bbox_inches='tight', facecolor='white')
            plt.close()

            if self.logger:
                self.logger.info(f"Visualización guardada: {nombre_vis}")

            return str(archivo_salida)

        except Exception as e:
            if self.logger:
                self.logger.error(f"Error generando visualización para {nombre_archivo}: {str(e)}")
            return None

    def _generar_visualizacion_mascaras(self, ax, imagen_original: Image.Image,
                                       datos_umbral: Dict, umbral_key: str):
        """Genera visualización de máscaras detectadas"""
        img_gray = np.array(imagen_original.convert('L'))
        h, w = img_gray.shape
        mask_combined = np.zeros_like(img_gray, dtype=np.float32)

        num_mascaras = 0
        analisis_mascaras = datos_umbral.get('analisis_mascaras_avanzado')

        if analisis_mascaras and analisis_mascaras.get('mascaras_individuales'):
            for i, mascara_data in enumerate(analisis_mascaras['mascaras_individuales']):
                if 'error' not in mascara_data:
                    geom = mascara_data.get('caracteristicas_geometricas', {})

                    # Crear máscara desde análisis
                    if 'centroid_y' in geom and 'centroid_x' in geom:
                        cy, cx = int(geom['centroid_y']), int(geom['centroid_x'])
                        area = geom.get('area_pixels', 0)

                        if area > 0:
                            radius = int(np.sqrt(area / np.pi))
                            y_grid, x_grid = np.ogrid[:h, :w]
                            mask_circle = (x_grid - cx)**2 + (y_grid - cy)**2 <= radius**2
                            mask_combined[mask_circle] += (i + 1) * 0.3
                            num_mascaras += 1

        # Visualizar máscaras
        if num_mascaras > 0:
            ax.imshow(img_gray, cmap='gray', alpha=0.7)
            mask_colored = np.ma.masked_where(mask_combined == 0, mask_combined)
            ax.imshow(mask_colored, cmap='Reds', alpha=0.8, vmin=0, vmax=1)
            ax.set_title(f'Máscaras Detectadas ({num_mascaras})', fontsize=14)
        else:
            ax.imshow(img_gray, cmap='gray')
            ax.set_title('No se detectaron personas', fontsize=14)

        ax.axis('off')

    def _generar_overlay(self, ax, imagen_original: Image.Image,
                        datos_umbral: Dict, umbral_key: str):
        """Genera overlay de imagen original con detecciones superpuestas"""
        img_array = np.array(imagen_original)
        analisis_mascaras = datos_umbral.get('analisis_mascaras_avanzado')
        contornos_dibujados = 0

        if analisis_mascaras and analisis_mascaras.get('mascaras_individuales'):
            for i, mascara_data in enumerate(analisis_mascaras['mascaras_individuales']):
                if 'error' not in mascara_data:
                    geom = mascara_data.get('caracteristicas_geometricas', {})
                    score = mascara_data.get('score_confianza', 0.0)

                    # Dibujar representación aproximada
                    if 'centroid_y' in geom and 'centroid_x' in geom:
                        cy, cx = int(geom['centroid_y']), int(geom['centroid_x'])
                        area = geom.get('area_pixels', 0)

                        if area > 0:
                            radius = int(np.sqrt(area / np.pi))

                            from matplotlib.patches import Circle
                            circle = Circle((cx, cy), radius, linewidth=3,
                                          edgecolor='red', facecolor='red', alpha=0.3)
                            ax.add_patch(circle)

                            ax.text(cx, cy-radius-10, f'Persona {i+1}\nScore: {score:.3f}',
                                   bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.8),
                                   fontsize=9, verticalalignment='top', ha='center')
                            contornos_dibujados += 1

        ax.imshow(img_array)
        personas = datos_umbral.get('personas_detectadas', 0)
        ax.set_title(f'Overlay - {personas} Persona(s) - {contornos_dibujados} Visualizadas', fontsize=14)
        ax.axis('off')

    def generar_resumen_visual(self, resultados_procesamiento: List[Dict],
                              nombre_modelo: str) -> Optional[str]:
        """Genera un resumen visual con múltiples imágenes procesadas"""
        try:
            exitosos = [r for r in resultados_procesamiento if r.get('metadatos', {}).get('exitoso', False)]
            if len(exitosos) == 0:
                return None

            muestra = exitosos[:12] if len(exitosos) > 12 else exitosos
            n_imgs = len(muestra)
            cols = min(4, n_imgs)
            rows = (n_imgs + cols - 1) // cols

            fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
            fig.suptitle(f'Resumen Visual - {nombre_modelo}', fontsize=16, fontweight='bold')

            if n_imgs == 1:
                axes_list = [axes]
            elif rows == 1:
                axes_list = axes if hasattr(axes, '__len__') else [axes]
            elif cols == 1:
                axes_list = axes if hasattr(axes, '__len__') else [axes]
            else:
                axes_list = axes.flatten()

            for i, resultado in enumerate(muestra):
                ax = axes_list[i]

                info_img = resultado['imagen']
                nombre_archivo = info_img['archivo']

                detecciones = resultado['deteccion'].get('detecciones_por_umbral', {})
                if detecciones:
                    primer_umbral = list(detecciones.values())[0]
                    personas = primer_umbral.get('personas_detectadas', 0)
                    score_max = primer_umbral.get('score_maximo', 0)

                    color = 'green' if personas > 0 else 'gray'

                    ax.text(0.5, 0.7, nombre_archivo, ha='center', va='center',
                           fontsize=10, weight='bold', transform=ax.transAxes)
                    ax.text(0.5, 0.5, f'Personas: {personas}', ha='center', va='center',
                           fontsize=12, color=color, weight='bold', transform=ax.transAxes)
                    ax.text(0.5, 0.3, f'Score máx: {score_max:.3f}', ha='center', va='center',
                           fontsize=10, transform=ax.transAxes)

                    ax.set_facecolor('lightgreen' if personas > 0 else 'lightgray')
                else:
                    ax.text(0.5, 0.5, f'{nombre_archivo}\nError de procesamiento',
                           ha='center', va='center', fontsize=10, transform=ax.transAxes)
                    ax.set_facecolor('lightcoral')

                ax.set_xticks([])
                ax.set_yticks([])
                ax.set_aspect('equal')

            # Ocultar axes sobrantes
            for i in range(n_imgs, len(axes_list)):
                axes_list[i].set_visible(False)

            # Estadísticas generales
            total_personas = sum(
                list(r['deteccion']['detecciones_por_umbral'].values())[0].get('personas_detectadas', 0)
                for r in exitosos if r['deteccion'].get('detecciones_por_umbral')
            )

            imagenes_con_personas = sum(
                1 for r in exitosos
                if r['deteccion'].get('detecciones_por_umbral') and
                list(r['deteccion']['detecciones_por_umbral'].values())[0].get('personas_detectadas', 0) > 0
            )

            fig.text(0.5, 0.02,
                    f'Total procesadas: {len(exitosos)} | Con personas: {imagenes_con_personas} | Total personas: {total_personas}',
                    ha='center', fontsize=12, style='italic')

            # Guardar resumen
            nombre_resumen = f"resumen_visual_{nombre_modelo}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
            archivo_resumen = self.directorio_salida / nombre_resumen

            plt.tight_layout()
            plt.savefig(archivo_resumen, dpi=150, bbox_inches='tight', facecolor='white')
            plt.close()

            if self.logger:
                self.logger.info(f"Resumen visual guardado: {nombre_resumen}")

            return str(archivo_resumen)

        except Exception as e:
            if self.logger:
                self.logger.error(f"Error generando resumen visual: {str(e)}")
            return None

# =============================================================================
# EXPORTADOR FORMATO COCO
# =============================================================================

class ExportadorCOCO:
    """Exportador de resultados a formato COCO"""

    def __init__(self, logger: logging.Logger = None):
        self.logger = logger
        self.coco_data = {
            "info": {
                "description": "Mask2Former Evaluation Results",
                "version": "1.0",
                "year": datetime.now().year,
                "contributor": "TFM Evaluation Framework",
                "date_created": datetime.now().isoformat()
            },
            "images": [],
            "annotations": [],
            "categories": [
                {
                    "id": 1,
                    "name": "person",
                    "supercategory": "person"
                }
            ]
        }
        self.image_id = 1
        self.annotation_id = 1

    def agregar_imagen(self, nombre_archivo: str, ancho: int, alto: int) -> int:
        """Agrega una imagen al dataset COCO"""
        imagen_info = {
            "id": self.image_id,
            "file_name": nombre_archivo,
            "width": ancho,
            "height": alto
        }
        self.coco_data["images"].append(imagen_info)

        current_id = self.image_id
        self.image_id += 1
        return current_id

    def agregar_anotacion(self, image_id: int, mascara: np.ndarray, score: float, bbox: List[int]):
        """Agrega una anotación de segmentación al formato COCO"""
        try:
            area = int(np.sum(mascara > 0.5))

            if area == 0:
                return

            anotacion = {
                "id": self.annotation_id,
                "image_id": image_id,
                "category_id": 1,  # person
                "bbox": bbox,  # [x, y, width, height]
                "area": area,
                "iscrowd": 0,
                "score": float(score)
            }

            self.coco_data["annotations"].append(anotacion)
            self.annotation_id += 1

        except Exception as e:
            if self.logger:
                self.logger.warning(f"Error agregando anotación COCO: {str(e)}")

    def exportar(self, archivo_salida: Path) -> None:
        """Exporta los datos en formato COCO"""
        try:
            with open(archivo_salida, 'w', encoding='utf-8') as f:
                json.dump(self.coco_data, f, indent=2, ensure_ascii=False)

            if self.logger:
                self.logger.info(f"Archivo COCO exportado: {archivo_salida.name}")
                self.logger.info(f"Imágenes: {len(self.coco_data['images'])}, Anotaciones: {len(self.coco_data['annotations'])}")

        except Exception as e:
            if self.logger:
                self.logger.error(f"Error exportando COCO: {str(e)}")



In [20]:
# =============================================================================
# UTILIDADES
# =============================================================================

class Utils:
    """Funciones utilitarias reutilizables"""

    @staticmethod
    def cargar_imagenes(ruta: str, extensiones: Tuple[str, ...] = (".jpg", ".png", ".jpeg")) -> List[str]:
        """Carga todas las imágenes desde una ruta recursivamente"""
        path = Path(ruta)
        imagenes = [
            str(p) for p in path.glob("**/*")
            if p.suffix.lower() in extensiones and p.is_file()
        ]
        return sorted(imagenes)

    @staticmethod
    def preparar_imagen(ruta: str, max_size: int = 1024) -> Image.Image:
        """Prepara una imagen para procesamiento"""
        try:
            img = Image.open(ruta).convert("RGB")
            if max(img.size) > max_size:
                img.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
            return img
        except Exception as e:
            raise ValueError(f"Error cargando imagen {ruta}: {str(e)}")

    @staticmethod
    def calcular_hash_imagen(ruta: str) -> str:
        """Calcula hash MD5 para identificación única de imagen"""
        try:
            with open(ruta, 'rb') as f:
                return hashlib.md5(f.read()).hexdigest()[:12]
        except Exception:
            return "hash_error"

    @staticmethod
    def guardar_json(datos: Any, archivo: Path, indent: int = 2) -> None:
        """Guarda datos en formato JSON con manejo de errores"""
        try:
            with open(archivo, 'w', encoding='utf-8') as f:
                json.dump(datos, f, indent=indent, ensure_ascii=False, default=str)
        except Exception as e:
            raise IOError(f"Error guardando JSON en {archivo}: {str(e)}")

    @staticmethod
    def crear_nombre_archivo(modelo_info: ModeloInfo, config_umbral: str,
                           timestamp: str, sanitizar: bool = False) -> str:
        """Crea nombres de archivo descriptivos y únicos"""
        if sanitizar:
            modelo_nombre = modelo_info.obtener_nombre_sanitizado()
            config_nombre = config_umbral[:8]
        else:
            modelo_nombre = f"{modelo_info.tipo}_{modelo_info.dataset.lower()}_{modelo_info.arquitectura.lower().replace('-', '')}"
            config_nombre = config_umbral

        return f"oneformer_{modelo_nombre}_{config_nombre}_{timestamp}.json"

In [22]:
# =============================================================================
# DETECTOR DE PERSONAS CON ONEFORMER
# =============================================================================

class DetectorPersonasOneFormer:
    """Detector de personas usando modelos OneFormer con análisis avanzado"""

    def __init__(self, modelo_info: ModeloInfo, logger: logging.Logger,
                 extractor, mask_analyzer, tarea_segmentacion: str = "instance"):
        self.modelo_info = modelo_info
        self.logger = logger
        self.extractor = extractor
        self.mask_analyzer = mask_analyzer
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # NUEVA: Tarea de segmentación
        self.tarea_segmentacion = tarea_segmentacion

        try:
            self.logger.info(f"Cargando modelo: {modelo_info.nombre_hf}")

            #OneFormer usa OneFormerProcessor
            self.processor = OneFormerProcessor.from_pretrained(modelo_info.nombre_hf)
            self.model = OneFormerForUniversalSegmentation.from_pretrained(modelo_info.nombre_hf)

            self.model.to(self.device)
            self.model.eval()
            self.logger.info(f"Modelo cargado exitosamente en {self.device}")
            self.logger.info(f"Tarea configurada: {self.tarea_segmentacion}")

            self._configurar_clases()

        except Exception as e:
            self.logger.error(f"Error cargando modelo: {str(e)}")
            raise

    def _configurar_clases(self) -> None:
        """Configura las clases y encuentra la clase 'persona'"""
        if hasattr(self.model.config, 'id2label'):
            self.id2label = self.model.config.id2label

            self.clase_persona = 0
            for clase_id, nombre in self.id2label.items():
                if any(term in nombre.lower() for term in ['person', 'people', 'human']):
                    self.clase_persona = clase_id
                    self.logger.info(f"Clase persona encontrada: {nombre} (ID: {clase_id})")
                    break
        else:
            self.clase_persona = 0
            self.logger.warning("No se encontró id2label, usando clase 0 por defecto")

    def detectar_en_imagen(self, imagen: Image.Image, umbrales: List[float]) -> Dict[str, Any]:
        """Ejecuta detección completa con análisis avanzado usando OneFormer"""
        inicio_tiempo = time.time()
        memoria_inicial = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0

        try:
            w, h = imagen.size

            #OneFormer requiere task_inputs
            # El processor de OneFormer procesa la imagen y la tarea
            inputs = self.processor(
                images=imagen,
                task_inputs=[self.tarea_segmentacion],  # NUEVO: Especificar tarea
                return_tensors="pt"
            ).to(self.device)

            # Inferencia
            inicio_inferencia = time.time()
            with torch.no_grad():
                outputs = self.model(**inputs)
            tiempo_inferencia_ms = (time.time() - inicio_inferencia) * 1000

            # Métricas de memoria
            memoria_maxima = torch.cuda.max_memory_allocated() if torch.cuda.is_available() else 0
            memoria_usada_mb = (memoria_maxima - memoria_inicial) / (1024 ** 2)

            resultados = {
                'timestamp': datetime.now().isoformat(),
                'modelo_info': asdict(self.modelo_info),
                'tarea_segmentacion': self.tarea_segmentacion,
                'rendimiento': {
                    'tiempo_inferencia_ms': tiempo_inferencia_ms,
                    'memoria_usada_mb': memoria_usada_mb,
                    'fps_estimado': 1000 / tiempo_inferencia_ms
                }
            }

            # Post-procesamiento según la tarea
            if self.tarea_segmentacion == 'semantic':
                resultados.update(self._procesar_segmentacion_semantica(outputs, (h, w), umbrales))
            elif self.tarea_segmentacion == 'instance':
                resultados.update(self._procesar_segmentacion_instancia(outputs, (h, w), umbrales))
            elif self.tarea_segmentacion == 'panoptic':
                resultados.update(self._procesar_segmentacion_panoptica(outputs, (h, w), umbrales))
            else:
                raise ValueError(f"Tarea no soportada: {self.tarea_segmentacion}")

            tiempo_total_ms = (time.time() - inicio_tiempo) * 1000
            resultados['rendimiento']['tiempo_total_ms'] = tiempo_total_ms

            return resultados

        except Exception as e:
            self.logger.error(f"Error en detección: {str(e)}")
            return self._crear_resultado_error(str(e), umbrales)

    def _procesar_segmentacion_semantica(self, outputs, target_size: Tuple[int, int],
                                        umbrales: List[float]) -> Dict:
        """Procesa resultados de segmentación semántica"""
        try:
            #OneFormer post-processing
            resultado_semantico = self.processor.post_process_semantic_segmentation(
                outputs, target_sizes=[target_size]
            )[0]

            unique_classes = torch.unique(resultado_semantico)
            total_pixels = resultado_semantico.numel()

            resultados_umbrales = {}

            for umbral in umbrales:
                persona_mask = (resultado_semantico == self.clase_persona)
                persona_pixels = persona_mask.sum().item()
                porcentaje_persona = (persona_pixels / total_pixels) * 100

                personas_detectadas = 1 if porcentaje_persona >= (umbral * 100) and persona_pixels > 50 else 0

                mask_for_storage = None
                if personas_detectadas > 0:
                    mask_for_storage = persona_mask.cpu().numpy().astype(np.float32)

                analisis_mascaras = None
                if personas_detectadas > 0:
                    analisis_mascaras = self.mask_analyzer.analizar_mascaras_completo(
                        [mask_for_storage], [1.0], target_size
                    )

                resultados_umbrales[f'umbral_{umbral}'] = {
                    'personas_detectadas': personas_detectadas,
                    'total_clases_detectadas': len(unique_classes),
                    'clases_presentes': unique_classes.tolist(),
                    'estadisticas_persona': {
                        'pixels_persona': persona_pixels,
                        'porcentaje_imagen': round(porcentaje_persona, 3),
                        'umbral_aplicado': umbral,
                        'criterio_cumplido': personas_detectadas > 0
                    },
                    '_mask_for_storage': mask_for_storage,
                    'analisis_mascaras_avanzado': analisis_mascaras,
                    'confianza_scores': [1.0] if personas_detectadas > 0 else [],
                    'score_maximo': 1.0 if len(unique_classes) > 0 else 0.0
                }

            return {'detecciones_por_umbral': resultados_umbrales}

        except Exception as e:
            self.logger.error(f"Error en segmentación semántica: {str(e)}")
            return self._crear_resultado_error(str(e), umbrales)

    def _procesar_segmentacion_instancia(self, outputs, target_size: Tuple[int, int],
                                        umbrales: List[float]) -> Dict:
        """Procesa resultados de segmentación de instancia"""
        resultados_umbrales = {}

        for umbral in umbrales:
            try:
                #OneFormer post-processing para instancias
                resultado = self.processor.post_process_instance_segmentation(
                    outputs, target_sizes=[target_size], threshold=umbral
                )[0]

                # Extraer segmentos
                segments_info = resultado.get("segments_info", [])
                segmentation_map = resultado.get("segmentation", None)

                # Filtrar personas
                personas_info = []
                masks_personas = []
                scores_personas = []

                for segment in segments_info:
                    if segment["label_id"] == self.clase_persona:
                        personas_info.append(segment)
                        scores_personas.append(segment["score"])

                        # Extraer máscara de este segmento
                        if segmentation_map is not None:
                            segment_mask = (segmentation_map == segment["id"]).cpu().numpy()
                            masks_personas.append(segment_mask.astype(np.float32))

                personas_detectadas = len(personas_info)

                # Análisis avanzado
                analisis_mascaras = None
                if personas_detectadas > 0 and masks_personas:
                    analisis_mascaras = self.mask_analyzer.analizar_mascaras_completo(
                        masks_personas, scores_personas, target_size
                    )

                resultados_umbrales[f'umbral_{umbral}'] = {
                    'personas_detectadas': personas_detectadas,
                    'total_detecciones': len(segments_info),
                    'segments_info': personas_info,
                    'estadisticas_persona': {
                        'numero_instancias': personas_detectadas,
                        'scores_individuales': scores_personas,
                        'score_promedio': np.mean(scores_personas) if scores_personas else 0.0,
                        'score_maximo_persona': max(scores_personas) if scores_personas else 0.0
                    },
                    '_masks_for_storage': masks_personas if masks_personas else None,
                    'analisis_mascaras_avanzado': analisis_mascaras,
                    'confianza_scores': scores_personas,
                    'score_maximo': max([s["score"] for s in segments_info]) if segments_info else 0.0
                }

            except Exception as e:
                self.logger.error(f"Error en umbral {umbral}: {str(e)}")
                resultados_umbrales[f'umbral_{umbral}'] = self._crear_resultado_error_umbral(str(e))

        return {'detecciones_por_umbral': resultados_umbrales}

    def _procesar_segmentacion_panoptica(self, outputs, target_size: Tuple[int, int],
                                        umbrales: List[float]) -> Dict:
        """Procesa resultados de segmentación panóptica"""
        resultados_umbrales = {}

        for umbral in umbrales:
            try:
                #OneFormer post-processing para panóptica
                resultado = self.processor.post_process_panoptic_segmentation(
                    outputs, target_sizes=[target_size], threshold=umbral
                )[0]

                segments_info = resultado.get("segments_info", [])
                segmentation_map = resultado.get("segmentation", None)

                # Filtrar personas (en panóptico pueden ser cosas o stuff)
                personas_info = []
                masks_personas = []
                scores_personas = []

                for segment in segments_info:
                    if segment["label_id"] == self.clase_persona:
                        personas_info.append(segment)
                        scores_personas.append(segment.get("score", 1.0))

                        if segmentation_map is not None:
                            segment_mask = (segmentation_map == segment["id"]).cpu().numpy()
                            masks_personas.append(segment_mask.astype(np.float32))

                personas_detectadas = len(personas_info)

                analisis_mascaras = None
                if personas_detectadas > 0 and masks_personas:
                    analisis_mascaras = self.mask_analyzer.analizar_mascaras_completo(
                        masks_personas, scores_personas, target_size
                    )

                resultados_umbrales[f'umbral_{umbral}'] = {
                    'personas_detectadas': personas_detectadas,
                    'total_segmentos': len(segments_info),
                    'segments_info': personas_info,
                    'estadisticas_persona': {
                        'numero_instancias': personas_detectadas,
                        'scores_individuales': scores_personas,
                        'score_promedio': np.mean(scores_personas) if scores_personas else 0.0,
                        'score_maximo_persona': max(scores_personas) if scores_personas else 0.0
                    },
                    '_masks_for_storage': masks_personas if masks_personas else None,
                    'analisis_mascaras_avanzado': analisis_mascaras,
                    'confianza_scores': scores_personas,
                    'score_maximo': max([s.get("score", 1.0) for s in segments_info]) if segments_info else 0.0
                }

            except Exception as e:
                self.logger.error(f"Error en umbral {umbral}: {str(e)}")
                resultados_umbrales[f'umbral_{umbral}'] = self._crear_resultado_error_umbral(str(e))

        return {'detecciones_por_umbral': resultados_umbrales}

    def _crear_resultado_error(self, error_msg: str, umbrales: List[float]) -> Dict:
        """Crea estructura de resultado para casos de error"""
        resultados_umbrales = {}
        for umbral in umbrales:
            resultados_umbrales[f'umbral_{umbral}'] = self._crear_resultado_error_umbral(error_msg)

        return {
            'error_general': error_msg,
            'detecciones_por_umbral': resultados_umbrales,
            'rendimiento': {'tiempo_inferencia_ms': 0, 'error': True}
        }

    def _crear_resultado_error_umbral(self, error_msg: str) -> Dict:
        """Crea resultado de error para un umbral específico"""
        return {
            'error': error_msg,
            'personas_detectadas': 0,
            'total_detecciones': 0,
            'confianza_scores': [],
            'score_maximo': 0.0,
            'analisis_mascaras_avanzado': None,
            '_mask_for_storage': None,
            '_masks_for_storage': None
        }

    def liberar_memoria(self) -> None:
        """Libera recursos del modelo"""
        self.logger.info("Liberando memoria del modelo...")
        if hasattr(self, 'model'):
            del self.model
        if hasattr(self, 'processor'):
            del self.processor

        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()

        self.logger.info("Memoria liberada")

In [23]:
# =============================================================================
# PROCESADOR DE RESULTADOS AVANZADO PARA ONEFORMER
# =============================================================================

class ProcesadorResultadosAvanzado:
    """Procesador completo con gestión separada de máscaras para OneFormer"""

    def __init__(self, directorio_salida: Path, logger_manager: LoggerManager):
        self.directorio_salida = directorio_salida
        self.logger_manager = logger_manager
        self.logger = logger_manager.crear_logger("procesador")

        # Inicializar componentes especializados
        self.extractor = ExtractorCaracteristicasAvanzado()
        self.extractor.set_logger(self.logger)
        self.mask_analyzer = MaskAnalyzer()

        # Gestor de máscaras separadas
        self.gestor_mascaras = GestorMascaras(directorio_salida, self.logger)

        # Inicializar generador de visualizaciones
        self.generador_vis = GeneradorVisualizaciones(
            directorio_salida / "visualizaciones",
            self.logger
        )

        # Contadores
        self.imagenes_procesadas = 0
        self.imagenes_exitosas = 0
        self.tiempo_total_procesamiento = 0

        # Lista para resumen visual
        self.resultados_para_resumen = []

        self.logger.info("Procesador de resultados OneFormer inicializado")

    def procesar_imagen(self, ruta_imagen: str, detector: DetectorPersonasOneFormer,
                       umbrales: List[float], exportador_coco: ExportadorCOCO = None,
                       modelo_nombre: str = "modelo", config_umbral_nombre: str = "config") -> Optional[Dict[str, Any]]:
        """Procesa una imagen con gestiÃ³n separada de máscaras"""
        inicio_procesamiento = time.time()
        nombre_archivo = os.path.basename(ruta_imagen)

        try:
            # Preparar imagen
            imagen = Utils.preparar_imagen(ruta_imagen)
            hash_imagen = Utils.calcular_hash_imagen(ruta_imagen)

            # Análisis avanzado de características
            caracteristicas_avanzadas = self.extractor.analizar_imagen_completa(imagen, ruta_imagen)

            # Información completa de la imagen
            info_imagen = {
                'archivo': nombre_archivo,
                'ruta_completa': ruta_imagen,
                'hash_md5': hash_imagen,
                'caracteristicas_avanzadas': caracteristicas_avanzadas,
                'timestamp_procesamiento': datetime.now().isoformat()
            }

            # Detección de personas con OneFormer
            resultados_deteccion = detector.detectar_en_imagen(imagen, umbrales)

            # Extraer y guardar máscaras separadamente
            archivo_mascaras = None
            if resultados_deteccion.get('detecciones_por_umbral'):
                mascaras_data = {}
                for umbral_key, umbral_data in resultados_deteccion['detecciones_por_umbral'].items():
                    mask_storage = umbral_data.pop('_mask_for_storage', None)
                    masks_storage = umbral_data.pop('_masks_for_storage', None)

                    mascaras_data[umbral_key] = {
                        'mascara_raw': mask_storage,
                        'mascaras_raw': masks_storage,
                        'confianza_scores': umbral_data.get('confianza_scores', [])
                    }

                archivo_mascaras = self.gestor_mascaras.guardar_mascaras_imagen(
                    nombre_archivo, mascaras_data, modelo_nombre, config_umbral_nombre
                )

            # Generar visualización
            archivo_visualizacion = self.generador_vis.generar_visualizacion_completa(
                imagen, nombre_archivo, resultados_deteccion, umbrales[0], modelo_nombre
            )

            # Exportar a COCO si se especifica
            if exportador_coco and resultados_deteccion.get('detecciones_por_umbral'):
                self._exportar_a_coco_desde_archivo(
                    exportador_coco, nombre_archivo, imagen.size,
                    archivo_mascaras, umbrales[0]
                )

            # Calcular métricas de procesamiento
            tiempo_total_ms = (time.time() - inicio_procesamiento) * 1000

            resultado_completo = {
                'metadatos': {
                    'timestamp': datetime.now().isoformat(),
                    'version_framework': '1.0_oneformer_universal',
                    'librerias_usadas': ['mahotas', 'scikit-image', 'shapely', 'sklearn'],
                    'framework': 'OneFormer',
                    'tarea_segmentacion': detector.tarea_segmentacion,
                    'exitoso': True
                },
                'imagen': info_imagen,
                'deteccion': resultados_deteccion,
                'mascaras': {
                    'archivo_mascaras': archivo_mascaras,
                    'almacenamiento': 'archivo_npz_separado',
                    'disponible': archivo_mascaras is not None
                },
                'visualizacion': {
                    'archivo_generado': archivo_visualizacion,
                    'disponible': archivo_visualizacion is not None
                },
                'rendimiento_total': {
                    'tiempo_procesamiento_completo_ms': tiempo_total_ms,
                    'memoria_sistema_mb': self._obtener_memoria_sistema()
                }
            }

            # Guardar para resumen visual
            self.resultados_para_resumen.append(resultado_completo)

            # Actualizar contadores
            self.imagenes_procesadas += 1
            self.imagenes_exitosas += 1
            self.tiempo_total_procesamiento += tiempo_total_ms

            self.logger.info(f"Imagen procesada exitosamente: {nombre_archivo}")

            return resultado_completo

        except Exception as e:
            self.logger_manager.log_error("procesador", e, f"Imagen: {nombre_archivo}")

            resultado_error = {
                'metadatos': {
                    'timestamp': datetime.now().isoformat(),
                    'version_framework': '1.0_oneformer_universal',
                    'framework': 'OneFormer',
                    'exitoso': False
                },
                'imagen': {'archivo': nombre_archivo, 'error': str(e)},
                'error_detalle': str(e),
                'mascaras': {
                    'archivo_mascaras': None,
                    'disponible': False
                },
                'visualizacion': {
                    'archivo_generado': None,
                    'disponible': False
                }
            }

            self.imagenes_procesadas += 1
            return resultado_error

    def finalizar_procesamiento(self, nombre_modelo: str) -> Optional[str]:
        """Genera resumen visual final y limpia recursos"""
        try:
            archivo_resumen_visual = self.generador_vis.generar_resumen_visual(
                self.resultados_para_resumen, nombre_modelo
            )

            self.resultados_para_resumen = []
            return archivo_resumen_visual

        except Exception as e:
            self.logger_manager.log_error("procesador", e, "Resumen visual final")
            return None

    def _exportar_a_coco_desde_archivo(self, exportador_coco: ExportadorCOCO,
                                      nombre_archivo: str, tamaño_imagen: Tuple[int, int],
                                      archivo_mascaras: str, umbral_ref: float):
        """Exporta a COCO cargando máscaras desde archivo separado"""
        try:
            if not archivo_mascaras:
                return

            ancho, alto = tamaño_imagen
            image_id = exportador_coco.agregar_imagen(nombre_archivo, ancho, alto)

            datos_mascaras = self.gestor_mascaras.cargar_mascaras_imagen(archivo_mascaras)

            if not datos_mascaras:
                return

            umbral_key = f'umbral_{umbral_ref}'
            if umbral_key in datos_mascaras['mascaras_por_umbral']:
                masks_list = datos_mascaras['mascaras_por_umbral'][umbral_key]
                metadatos = datos_mascaras['metadatos']
                scores = metadatos.get(f'{umbral_key}_scores', [1.0])

                for i, mask_array in enumerate(masks_list):
                    score = scores[i] if i < len(scores) else 1.0
                    bbox = self._calcular_bbox_desde_mascara(mask_array)
                    if bbox:
                        exportador_coco.agregar_anotacion(image_id, mask_array, score, bbox)

        except Exception as e:
            self.logger_manager.log_error("procesador", e, f"COCO export {nombre_archivo}")

    def _calcular_bbox_desde_mascara(self, mask: np.ndarray) -> Optional[List[int]]:
        """Calcula bounding box desde máscara binaria"""
        try:
            mask_binary = (mask > 0.5).astype(np.uint8)
            contours, _ = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if contours:
                contour = max(contours, key=cv2.contourArea)
                x, y, w, h = cv2.boundingRect(contour)
                return [int(x), int(y), int(w), int(h)]

            return None

        except Exception:
            return None

    def _obtener_memoria_sistema(self) -> float:
        """Obtiene el uso actual de memoria del sistema"""
        try:
            return psutil.Process().memory_info().rss / (1024 ** 2)
        except:
            return 0.0

    def generar_resumen_estadistico_avanzado(self, resultados: List[Dict], config_umbral: ConfiguracionUmbrales,
                                           modelo_info: ModeloInfo) -> Dict[str, Any]:
        """Genera resumen estadístico completo con análisis avanzado"""
        exitosos = [r for r in resultados if r.get('metadatos', {}).get('exitoso', False)]

        resumen = {
            'metadatos_resumen': {
                'timestamp': datetime.now().isoformat(),
                'modelo_evaluado': asdict(modelo_info),
                'configuracion_umbrales': asdict(config_umbral),
                'total_imagenes': len(resultados),
                'imagenes_exitosas': len(exitosos),
                'tasa_exito': len(exitosos) / len(resultados) if resultados else 0,
                'version_framework': '1.0_oneformer_universal',
                'framework': 'OneFormer',
                'tarea_segmentacion': resultados[0]['metadatos'].get('tarea_segmentacion', 'unknown') if resultados else 'unknown'
            },
            'estadisticas_rendimiento': self._calcular_estadisticas_rendimiento(exitosos),
            'estadisticas_deteccion': self._calcular_estadisticas_deteccion(exitosos, config_umbral.valores),
            'estadisticas_caracteristicas_avanzadas': self._analizar_caracteristicas_avanzadas(exitosos),
            'estadisticas_mascaras_shapely': self._analizar_estadisticas_shapely(exitosos),
            'distribucion_caracteristicas': self._analizar_caracteristicas_imagenes(exitosos)
        }

        return resumen

    def _calcular_estadisticas_rendimiento(self, resultados_exitosos: List[Dict]) -> Dict[str, Any]:
        """Calcula estadísticas de rendimiento del sistema"""
        if not resultados_exitosos:
            return {}

        tiempos_inferencia = []
        memoria_usada = []

        for resultado in resultados_exitosos:
            rendimiento = resultado.get('deteccion', {}).get('rendimiento', {})
            if 'tiempo_inferencia_ms' in rendimiento:
                tiempos_inferencia.append(rendimiento['tiempo_inferencia_ms'])
            if 'memoria_usada_mb' in rendimiento:
                memoria_usada.append(rendimiento['memoria_usada_mb'])

        estadisticas = {
            'tiempo_inferencia': {
                'promedio_ms': np.mean(tiempos_inferencia) if tiempos_inferencia else 0,
                'mediana_ms': np.median(tiempos_inferencia) if tiempos_inferencia else 0,
                'minimo_ms': np.min(tiempos_inferencia) if tiempos_inferencia else 0,
                'maximo_ms': np.max(tiempos_inferencia) if tiempos_inferencia else 0,
                'desviacion_std': np.std(tiempos_inferencia) if tiempos_inferencia else 0
            }
        }

        if memoria_usada:
            estadisticas['memoria'] = {
                'promedio_mb': np.mean(memoria_usada),
                'maxima_mb': np.max(memoria_usada),
                'minima_mb': np.min(memoria_usada)
            }

        return estadisticas

    def _calcular_estadisticas_deteccion(self, resultados_exitosos: List[Dict], umbrales: List[float]) -> Dict[str, Any]:
        """Calcula estadísticas de detección por umbral"""
        if not resultados_exitosos:
            return {}

        estadisticas_por_umbral = {}

        for umbral in umbrales:
            personas_detectadas = []
            scores_maximos = []

            for resultado in resultados_exitosos:
                detecciones = resultado.get('deteccion', {}).get('detecciones_por_umbral', {})
                umbral_data = detecciones.get(f'umbral_{umbral}', {})

                if 'error' not in umbral_data:
                    personas_detectadas.append(umbral_data.get('personas_detectadas', 0))
                    scores_maximos.append(umbral_data.get('score_maximo', 0))

            if personas_detectadas:
                total_personas = sum(personas_detectadas)
                imagenes_con_personas = sum(1 for p in personas_detectadas if p > 0)

                estadisticas_por_umbral[f'umbral_{umbral}'] = {
                    'total_personas_detectadas': total_personas,
                    'imagenes_con_personas': imagenes_con_personas,
                    'imagenes_sin_personas': len(personas_detectadas) - imagenes_con_personas,
                    'porcentaje_imagenes_con_personas': (imagenes_con_personas / len(personas_detectadas)) * 100,
                    'personas_promedio_por_imagen': total_personas / len(personas_detectadas),
                    'score_promedio': np.mean(scores_maximos) if scores_maximos else 0,
                    'score_mediano': np.median(scores_maximos) if scores_maximos else 0
                }

        return estadisticas_por_umbral

    def _analizar_caracteristicas_avanzadas(self, resultados_exitosos: List[Dict]) -> Dict[str, Any]:
        """Analiza las características avanzadas extraídas"""
        if not resultados_exitosos:
            return {}

        haralick_means = []
        glcm_contrasts = []
        num_corners = []
        zernike_means = []
        num_regiones = []

        for resultado in resultados_exitosos:
            carac = resultado.get('imagen', {}).get('caracteristicas_avanzadas', {})

            texturas_mh = carac.get('texturas_mahotas', {})
            if 'haralick_features' in texturas_mh and texturas_mh['haralick_features']:
                haralick_means.append(np.mean(texturas_mh['haralick_features']))

            if 'zernike_moments' in texturas_mh and texturas_mh['zernike_moments']:
                zernike_means.append(np.mean(texturas_mh['zernike_moments']))

            texturas_sk = carac.get('texturas_skimage', {})
            glcm_props = texturas_sk.get('glcm_properties', {})
            if 'contrast' in glcm_props:
                glcm_contrasts.append(glcm_props['contrast'])

            geom = carac.get('caracteristicas_geometricas', {})
            if 'num_corners' in geom:
                num_corners.append(geom['num_corners'])

            regionales = carac.get('propiedades_regionales', {})
            if 'num_regiones' in regionales:
                num_regiones.append(regionales['num_regiones'])

        def safe_stats(values, nombre):
            if not values:
                return {'count': 0, 'mean': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0}
            return {
                'count': len(values),
                'mean': float(np.mean(values)),
                'std': float(np.std(values)),
                'min': float(np.min(values)),
                'max': float(np.max(values))
            }

        return {
            'haralick_estadisticas': safe_stats(haralick_means, 'Haralick'),
            'glcm_contrast_estadisticas': safe_stats(glcm_contrasts, 'GLCM Contrast'),
            'corners_estadisticas': safe_stats(num_corners, 'Corners'),
            'zernike_estadisticas': safe_stats(zernike_means, 'Zernike'),
            'regiones_estadisticas': safe_stats(num_regiones, 'Regiones'),
            'imagenes_con_caracteristicas': len([r for r in resultados_exitosos
                                               if r.get('imagen', {}).get('caracteristicas_avanzadas')])
        }

    def _analizar_estadisticas_shapely(self, resultados_exitosos: List[Dict]) -> Dict[str, Any]:
        """Analiza estadísticas específicas de análisis con Shapely"""
        if not resultados_exitosos:
            return {}

        compactness_values = []
        convexity_ratios = []
        roughness_factors = []
        num_vertices = []
        mascaras_validas = 0
        total_mascaras = 0

        for resultado in resultados_exitosos:
            detecciones = resultado.get('deteccion', {}).get('detecciones_por_umbral', {})

            for umbral_key, umbral_data in detecciones.items():
                analisis_mascaras = umbral_data.get('analisis_mascaras_avanzado')
                if analisis_mascaras and analisis_mascaras.get('mascaras_individuales'):
                    for mascara_data in analisis_mascaras['mascaras_individuales']:
                        total_mascaras += 1

                        if 'error' not in mascara_data:
                            geom = mascara_data.get('caracteristicas_geometricas', {})
                            if 'compactness' in geom:
                                compactness_values.append(geom['compactness'])

                            shapely_geom = mascara_data.get('shapely_geometry', {})
                            if shapely_geom.get('polygon_valido'):
                                mascaras_validas += 1
                                if 'convexity_ratio' in shapely_geom:
                                    convexity_ratios.append(shapely_geom['convexity_ratio'])
                                if 'num_vertices' in shapely_geom:
                                    num_vertices.append(shapely_geom['num_vertices'])

                            forma = mascara_data.get('caracteristicas_forma', {})
                            if 'roughness_factor' in forma:
                                roughness_factors.append(forma['roughness_factor'])

        def safe_stats(values):
            if not values:
                return {'count': 0, 'mean': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0}
            return {
                'count': len(values),
                'mean': float(np.mean(values)),
                'std': float(np.std(values)),
                'min': float(np.min(values)),
                'max': float(np.max(values))
            }

        return {
            'total_mascaras_analizadas': total_mascaras,
            'mascaras_shapely_validas': mascaras_validas,
            'tasa_validez_shapely': mascaras_validas / total_mascaras if total_mascaras > 0 else 0,
            'compactness_estadisticas': safe_stats(compactness_values),
            'convexity_ratio_estadisticas': safe_stats(convexity_ratios),
            'roughness_estadisticas': safe_stats(roughness_factors),
            'vertices_estadisticas': safe_stats(num_vertices)
        }

    def _analizar_caracteristicas_imagenes(self, resultados_exitosos: List[Dict]) -> Dict[str, Any]:
        """Analiza la distribución de características de las imágenes procesadas"""
        if not resultados_exitosos:
            return {}

        anchos, altos, brillos, contrastes = [], [], [], []
        megapixeles, aspect_ratios = [], []

        for resultado in resultados_exitosos:
            carac_avanzadas = resultado.get('imagen', {}).get('caracteristicas_avanzadas', {})

            metadatos = carac_avanzadas.get('metadatos_basicos', {})
            if metadatos:
                dims = metadatos.get('dimensiones', {})
                if dims:
                    anchos.append(dims.get('ancho', 0))
                    altos.append(dims.get('alto', 0))

                if 'megapixeles' in metadatos:
                    megapixeles.append(metadatos['megapixeles'])
                if 'aspecto_ratio' in metadatos:
                    aspect_ratios.append(metadatos['aspecto_ratio'])

            color = carac_avanzadas.get('color_y_paleta', {})
            if color:
                lab = color.get('lab_luminancia', {})
                if 'L_medio' in lab:
                    brillos.append(lab['L_medio'])

                rgb_stats = color.get('estadisticas_rgb', {})
                if 'std' in rgb_stats and rgb_stats['std']:
                    contrastes.append(np.mean(rgb_stats['std']))

        def safe_stats(values):
            if not values:
                return {'mean': 0.0, 'std': 0.0, 'min': 0.0, 'max': 0.0}
            return {
                'mean': float(np.mean(values)),
                'std': float(np.std(values)),
                'min': float(np.min(values)),
                'max': float(np.max(values))
            }

        return {
            'dimensiones': {
                'ancho_estadisticas': safe_stats(anchos),
                'alto_estadisticas': safe_stats(altos),
                'megapixeles_estadisticas': safe_stats(megapixeles),
                'aspect_ratio_estadisticas': safe_stats(aspect_ratios)
            },
            'iluminacion_color': {
                'brillo_lab_estadisticas': safe_stats(brillos),
                'contraste_rgb_estadisticas': safe_stats(contrastes)
            },
            'resumen_dataset': {
                'imagenes_analizadas': len(resultados_exitosos),
                'resolucion_promedio_mp': np.mean(megapixeles) if megapixeles else 0,
                'orientacion_predominante': 'horizontal' if np.mean(aspect_ratios) > 1 else 'vertical' if aspect_ratios else 'desconocida'
            }
        }

    def mostrar_resumen_consola_avanzado(self, resumen: Dict[str, Any]) -> None:
        """Muestra un resumen completo en consola con características avanzadas"""
        meta = resumen['metadatos_resumen']
        modelo = meta['modelo_evaluado']

        print(f"\n{'='*100}")
        print(f"RESUMEN DE EVALUACION AVANZADA - ONEFORMER")
        print(f"{'='*100}")
        print(f"Modelo: {modelo['nombre_corto']}")
        print(f"Dataset: {modelo['dataset']} | Tipo: {modelo['tipo'].upper()}")
        print(f"Tarea: {meta['tarea_segmentacion'].upper()}")
        print(f"Éxito: {meta['imagenes_exitosas']}/{meta['total_imagenes']} ({meta['tasa_exito']*100:.1f}%)")
        print(f"Framework: {meta['framework']}")

        rendimiento = resumen.get('estadisticas_rendimiento', {})
        if 'tiempo_inferencia' in rendimiento:
            tiempo = rendimiento['tiempo_inferencia']
            print(f"Tiempo promedio: {tiempo['promedio_ms']:.1f}ms (min: {tiempo['minimo_ms']:.1f}ms, max: {tiempo['maximo_ms']:.1f}ms)")

        deteccion = resumen.get('estadisticas_deteccion', {})
        if deteccion:
            print(f"\nRESULTADOS POR UMBRAL:")
            for umbral_key, stats in deteccion.items():
                umbral_val = umbral_key.replace('umbral_', '')
                print(f"   {umbral_val:>8}: {stats['imagenes_con_personas']:3d}/{meta['imagenes_exitosas']} imágenes ({stats['porcentaje_imagenes_con_personas']:5.1f}%) | {stats['total_personas_detectadas']:3d} personas")

        carac_avanzadas = resumen.get('estadisticas_caracteristicas_avanzadas', {})
        if carac_avanzadas and carac_avanzadas.get('imagenes_con_caracteristicas', 0) > 0:
            print(f"\nANALISIS DE CARACTERISTICAS AVANZADAS:")

            haralick = carac_avanzadas.get('haralick_estadisticas', {})
            if haralick.get('count', 0) > 0:
                print(f"   Haralick (textura): {haralick['mean']:.3f} ± {haralick['std']:.3f}")

            corners = carac_avanzadas.get('corners_estadisticas', {})
            if corners.get('count', 0) > 0:
                print(f"   Esquinas detectadas: {corners['mean']:.1f} ± {corners['std']:.1f}")

            glcm = carac_avanzadas.get('glcm_contrast_estadisticas', {})
            if glcm.get('count', 0) > 0:
                print(f"   GLCM Contraste: {glcm['mean']:.3f} ± {glcm['std']:.3f}")

        shapely_stats = resumen.get('estadisticas_mascaras_shapely', {})
        if shapely_stats and shapely_stats.get('total_mascaras_analizadas', 0) > 0:
            print(f"\nANALISIS GEOMETRICO SHAPELY:")
            print(f"   Máscaras analizadas: {shapely_stats['total_mascaras_analizadas']}")
            print(f"   Polígonos válidos: {shapely_stats['mascaras_shapely_validas']} ({shapely_stats['tasa_validez_shapely']*100:.1f}%)")

            compactness = shapely_stats.get('compactness_estadisticas', {})
            if compactness.get('count', 0) > 0:
                print(f"   Compacidad promedio: {compactness['mean']:.3f} ± {compactness['std']:.3f}")

            convexity = shapely_stats.get('convexity_ratio_estadisticas', {})
            if convexity.get('count', 0) > 0:
                print(f"   Ratio convexidad: {convexity['mean']:.3f} ± {convexity['std']:.3f}")

In [24]:
# =============================================================================
# EVALUADOR PRINCIPAL ONEFORMER AVANZADO
# =============================================================================

class EvaluadorOneFormerAvanzado:
    """Clase principal que orquesta todo el proceso de evaluación avanzado con OneFormer"""

    def __init__(self, config: ConfigEvaluacion):
        self.config = config

        # Crear directorio de ejecución
        self.directorio_ejecucion = config.crear_directorio_ejecucion()
        print(f"\n📁 Directorio de ejecución: {self.directorio_ejecucion}")

        # Inicializar sistema de logging
        self.logger_manager = LoggerManager(self.directorio_ejecucion / "logs")
        self.logger = self.logger_manager.crear_logger("evaluador_principal")

        # Inicializar procesador avanzado
        self.procesador = ProcesadorResultadosAvanzado(self.directorio_ejecucion, self.logger_manager)

        self.logger.info("Evaluador OneFormer principal inicializado")
        self.logger.info(f"Tarea de segmentación: {config.TAREA_SEGMENTACION}")

    def ejecutar_evaluacion_modelo(self, idx_modelo: int, nombre_config_umbral: str) -> Optional[str]:
        """Ejecuta evaluación completa para un modelo y configuración específica"""
        # Validaciones
        if idx_modelo >= len(self.config.MODELOS):
            self.logger.error(f"Índice de modelo inválido: {idx_modelo}")
            return None

        if nombre_config_umbral not in self.config.UMBRALES:
            self.logger.error(f"Configuración de umbral inválida: {nombre_config_umbral}")
            return None

        # Configuración actual
        modelo_info = self.config.MODELOS[idx_modelo]
        config_umbral = self.config.UMBRALES[nombre_config_umbral]

        self.logger.info(f"Iniciando evaluación: {modelo_info.nombre_corto} - {nombre_config_umbral}")
        self.logger.info(f"Tarea: {self.config.TAREA_SEGMENTACION}")

        try:
            # Cargar dataset
            imagenes = Utils.cargar_imagenes(str(self.config.DATASET_PATH))
            if not imagenes:
                self.logger.error("No se encontraron imágenes en el dataset")
                return None

            self.logger.info(f"Dataset cargado: {len(imagenes)} imágenes encontradas")

            # Limitar imágenes si es necesario
            if len(imagenes) > self.config.MAX_IMAGENES_LOTE:
                imagenes = imagenes[:self.config.MAX_IMAGENES_LOTE]
                self.logger.info(f"Limitando a {self.config.MAX_IMAGENES_LOTE} imágenes")

            # CAMBIO IMPORTANTE: Inicializar DetectorPersonasOneFormer
            detector_logger = self.logger_manager.crear_logger(f"detector_{modelo_info.nombre_corto}")
            detector = DetectorPersonasOneFormer(
                modelo_info,
                detector_logger,
                self.procesador.extractor,
                self.procesador.mask_analyzer,
                self.config.TAREA_SEGMENTACION  # Pasar tarea de segmentación
            )

            # Inicializar exportador COCO si está habilitado
            exportador_coco = None
            if self.config.GENERAR_FORMATO_COCO:
                exportador_coco = ExportadorCOCO(self.logger)

            # Nombres para archivos
            if self.config.USAR_NOMBRES_SANITIZADOS:
                modelo_nombre = modelo_info.obtener_nombre_sanitizado()
                config_nombre = config_umbral.obtener_nombre_sanitizado()
            else:
                modelo_nombre = modelo_info.nombre_corto
                config_nombre = config_umbral.nombre

            # Procesar imágenes
            resultados = []
            tiempo_inicio_total = time.time()

            self.logger.info(f"Procesando {len(imagenes)} imágenes...")

            for i, ruta_imagen in enumerate(tqdm(imagenes, desc=f"Procesando {modelo_info.nombre_corto}"), 1):
                # Limpiar caché periódicamente
                if i % self.config.LIMPIAR_CACHE_CADA == 0:
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    self.logger.info(f"Caché GPU limpiado en imagen {i}")

                resultado = self.procesador.procesar_imagen(
                    ruta_imagen, detector, config_umbral.valores, exportador_coco,
                    modelo_nombre, config_nombre
                )

                if resultado:
                    resultados.append(resultado)

            tiempo_total_s = time.time() - tiempo_inicio_total
            self.logger.info(f"Procesamiento completado en {tiempo_total_s:.1f} segundos")

            # Generar resumen estadístico avanzado
            resumen = self.procesador.generar_resumen_estadistico_avanzado(
                resultados, config_umbral, modelo_info
            )

            # Generar resumen visual final
            archivo_resumen_visual = self.procesador.finalizar_procesamiento(modelo_nombre)

            # Guardar resultados
            archivo_generado = self._guardar_resultados_simplificado(
                resultados, resumen, modelo_info, config_umbral, exportador_coco,
                archivo_resumen_visual, modelo_nombre, config_nombre
            )

            # Mostrar resumen en consola
            self.procesador.mostrar_resumen_consola_avanzado(resumen)

            # Mostrar estadísticas de almacenamiento de máscaras
            stats_mascaras = self.procesador.gestor_mascaras.obtener_estadisticas_almacenamiento()
            if stats_mascaras['total_archivos'] > 0:
                print(f"\nESTADÍSTICAS DE MÁSCARAS:")
                print(f"   Archivos NPZ generados: {stats_mascaras['total_archivos']}")
                print(f"   Tamaño total: {stats_mascaras['tamaño_total_mb']} MB")
                print(f"   Tamaño promedio: {stats_mascaras['tamaño_promedio_mb']} MB")

            # Liberar memoria del detector
            detector.liberar_memoria()

            self.logger.info(f"Evaluación completada: {archivo_generado}")

            return archivo_generado

        except Exception as e:
            self.logger.error(f"Error en evaluación: {str(e)}")
            import traceback
            self.logger.error(traceback.format_exc())
            return None

    def _guardar_resultados_simplificado(self, resultados: List[Dict], resumen: Dict,
                                       modelo_info: ModeloInfo, config_umbral: ConfiguracionUmbrales,
                                       exportador_coco: ExportadorCOCO = None,
                                       archivo_resumen_visual: str = None,
                                       modelo_nombre: str = "modelo",
                                       config_nombre: str = "config") -> Optional[str]:
        """Guarda resultados en formatos esenciales con nombres configurables"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        nombre_archivo = f"{modelo_nombre}_{config_nombre}_{timestamp}.json"

        try:
            # Crear directorio específico del modelo
            directorio_modelo = self.directorio_ejecucion / modelo_nombre
            directorio_modelo.mkdir(exist_ok=True)

            # Archivo JSON principal
            datos_completos = {
                'metadatos_archivo': {
                    'timestamp_creacion': datetime.now().isoformat(),
                    'version_framework': '1.0_oneformer_universal',
                    'framework': 'OneFormer',
                    'tarea_segmentacion': self.config.TAREA_SEGMENTACION,
                    'modelo_evaluado': asdict(modelo_info),
                    'configuracion_umbrales': asdict(config_umbral),
                    'total_imagenes_procesadas': len(resultados),
                    'librerias_usadas': ['mahotas', 'scikit-image', 'shapely', 'sklearn', 'opencv-python'],
                    'mascaras_raw_incluidas': False,
                    'mascaras_almacenamiento': 'archivos_npz_separados'
                },
                'resumen_estadistico': resumen,
                'resultados_detallados': resultados
            }

            archivo_principal = directorio_modelo / nombre_archivo
            Utils.guardar_json(datos_completos, archivo_principal)
            self.logger.info(f"JSON principal guardado: {archivo_principal}")

            # Mover visualización al directorio del modelo si existe
            if archivo_resumen_visual and self.config.GUARDAR_VISUALIZACIONES:
                try:
                    import shutil
                    resumen_destino = directorio_modelo / Path(archivo_resumen_visual).name
                    if Path(archivo_resumen_visual).exists():
                        shutil.move(archivo_resumen_visual, resumen_destino)
                        self.logger.info(f"Resumen visual movido: {resumen_destino}")
                except Exception as e:
                    self.logger.error(f"Error moviendo visualización: {str(e)}")

            # Exportar COCO si está habilitado
            if exportador_coco and self.config.GENERAR_FORMATO_COCO:
                try:
                    nombre_coco = f"{modelo_nombre}_{config_nombre}_{timestamp}_coco.json"
                    archivo_coco = self.directorio_ejecucion / "formato_coco" / nombre_coco
                    exportador_coco.exportar(archivo_coco)
                    self.logger.info(f"Archivo COCO exportado: {archivo_coco}")
                except Exception as e:
                    self.logger.error(f"Error exportando COCO: {str(e)}")

            return str(archivo_principal)

        except Exception as e:
            self.logger.error(f"Error guardando resultados: {str(e)}")
            return None

    def ejecutar_evaluacion_completa(self) -> Dict[str, List[str]]:
        """Ejecuta evaluación completa de todos los modelos con todas las configuraciones"""
        total_combinaciones = len(self.config.MODELOS) * len(self.config.UMBRALES)
        archivos_generados = {}

        print(f"\n{'='*100}")
        print(f"EVALUACION COMPLETA ONEFORMER: {total_combinaciones} combinaciones")
        print(f"Tarea: {self.config.TAREA_SEGMENTACION.upper()}")
        print(f"Análisis incluidos: Texturas, Geometría, Color, Forma, Multi-escala")
        print(f"Máscaras: Almacenamiento separado en archivos NPZ comprimidos")
        print(f"{'='*100}")

        self.logger.info(f"Iniciando evaluación completa: {total_combinaciones} combinaciones")

        combinacion_actual = 0

        try:
            for i, modelo_info in enumerate(self.config.MODELOS):
                archivos_modelo = []

                for nombre_config in self.config.UMBRALES.keys():
                    combinacion_actual += 1

                    print(f"\n{'='*100}")
                    print(f"COMBINACION {combinacion_actual}/{total_combinaciones}")
                    print(f"Modelo: {modelo_info.nombre_corto} | Config: {nombre_config}")
                    print(f"Tarea: {self.config.TAREA_SEGMENTACION}")
                    print(f"{'='*100}")

                    self.logger.info(f"Combinación {combinacion_actual}/{total_combinaciones}: {modelo_info.nombre_corto} - {nombre_config}")

                    archivo_resultado = self.ejecutar_evaluacion_modelo(i, nombre_config)

                    if archivo_resultado:
                        archivos_modelo.append(archivo_resultado)
                        print(f"✅ Combinación {combinacion_actual} completada exitosamente")
                        self.logger.info(f"Combinación {combinacion_actual} completada")
                    else:
                        print(f"❌ Error en combinación {combinacion_actual}")
                        self.logger.error(f"Error en combinación {combinacion_actual}")

                    # Pausa entre combinaciones
                    if combinacion_actual < total_combinaciones:
                        time.sleep(3)

                archivos_generados[modelo_info.nombre_corto] = archivos_modelo

            print(f"\n{'='*100}")
            print(f"EVALUACION COMPLETA ONEFORMER FINALIZADA")
            print(f"{'='*100}")
            print(f"Todos los resultados en: {self.directorio_ejecucion}")
            print(f"Análisis completo con librerías especializadas")
            print(f"Máscaras almacenadas en archivos NPZ separados")

            self.logger.info("Evaluación completa finalizada exitosamente")

            # Limpiar logs vacíos al final
            self.logger_manager.limpiar_logs_vacios()

            # Cerrar todos los loggers
            self.logger_manager.cerrar_loggers()

            return archivos_generados

        except KeyboardInterrupt:
            print(f"\n⚠️ EVALUACION INTERRUMPIDA POR USUARIO")
            print(f"Progreso: {combinacion_actual}/{total_combinaciones}")
            self.logger.warning(f"Evaluación interrumpida en combinación {combinacion_actual}/{total_combinaciones}")

            self.logger_manager.cerrar_loggers()
            return archivos_generados

        except Exception as e:
            self.logger.error(f"Error crítico en evaluación completa: {str(e)}")
            import traceback
            self.logger.error(traceback.format_exc())

            self.logger_manager.cerrar_loggers()
            return archivos_generados

    def obtener_estadisticas_ejecucion(self) -> Dict[str, Any]:
        """Obtiene estadísticas de la ejecución actual"""
        try:
            stats_logs = self.logger_manager.obtener_estadisticas_logs()

            # Contar archivos generados por modelo
            archivos_por_modelo = {}
            for modelo_dir in self.directorio_ejecucion.iterdir():
                if modelo_dir.is_dir() and modelo_dir.name not in ["logs", "visualizaciones", "formato_coco", "mascaras_raw"]:
                    archivos_json = list(modelo_dir.glob("*.json"))
                    archivos_por_modelo[modelo_dir.name] = len(archivos_json)

            # Estadísticas de máscaras
            stats_mascaras = self.procesador.gestor_mascaras.obtener_estadisticas_almacenamiento()

            return {
                'directorio_ejecucion': str(self.directorio_ejecucion),
                'timestamp_ejecucion': self.directorio_ejecucion.name,
                'framework': 'OneFormer',
                'tarea_segmentacion': self.config.TAREA_SEGMENTACION,
                'modelos_procesados': len(archivos_por_modelo),
                'archivos_por_modelo': archivos_por_modelo,
                'total_archivos_json': sum(archivos_por_modelo.values()),
                'estadisticas_logs': stats_logs,
                'estadisticas_mascaras': stats_mascaras,
                'imagenes_procesadas': self.procesador.imagenes_procesadas,
                'imagenes_exitosas': self.procesador.imagenes_exitosas,
                'tiempo_total_procesamiento_ms': self.procesador.tiempo_total_procesamiento
            }

        except Exception as e:
            self.logger.error(f"Error obteniendo estadísticas: {str(e)}")
            return {}

In [30]:
def main():
    """
    Función principal única del sistema de evaluación OneFormer
    Ejecuta automáticamente todos los modelos con todas las configuraciones
    """
    try:
        # Configuración inicial
        config = ConfigEvaluacion()

        print(f"\n{'='*100}")
        print(f"SISTEMA DE EVALUACION ONEFORMER (UNIVERSAL SEGMENTATION)")
        print(f"{'='*100}")

        print(f"\nTarea de segmentación: {config.TAREA_SEGMENTACION}")
        print(f"\nConfiguración de evaluación:")
        print(f"   Modelos: {len(config.MODELOS)}")
        for i, modelo in enumerate(config.MODELOS):
            print(f"      [{i}] {modelo.nombre_corto} ({modelo.tipo}, {modelo.dataset})")
            print(f"          Tareas: {', '.join(modelo.tareas_soportadas)}")

        print(f"   Configuraciones de umbral: {len(config.UMBRALES)}")
        for nombre, config_umbral in config.UMBRALES.items():
            print(f"      {nombre}: {config_umbral.descripcion}")

        # Validar configuración
        if not config.validar_configuracion():
            print("❌ ERROR: Configuración inválida. Verificar paths y parámetros.")
            return

        print("✅ Configuración validada correctamente")

        # Crear evaluador
        evaluador = EvaluadorOneFormerAvanzado(config)

        # Mostrar resumen de configuración
        resumen_config = config.obtener_resumen_configuracion()
        print(f"\nRESUMEN DE CONFIGURACION:")
        print(f"   Total combinaciones: {resumen_config['total_combinaciones']}")
        print(f"   Dataset: {resumen_config['dataset_path']}")
        print(f"   Visualizaciones: {'✓ Habilitadas' if resumen_config['visualizaciones_habilitadas'] else '✗ Deshabilitadas'}")
        print(f"   Formato COCO: {'✓ Habilitado' if resumen_config['formato_coco_habilitado'] else '✗ Deshabilitado'}")
        print(f"   Max imágenes por lote: {resumen_config['max_imagenes_lote']}")
        print(f"   Framework: {resumen_config['framework']}")

        # Ejecutar evaluación completa automáticamente
        print(f"\n🚀 INICIANDO EVALUACION AUTOMATICA COMPLETA...")
        archivos_resultado = evaluador.ejecutar_evaluacion_completa()

        # Obtener estadísticas finales
        estadisticas = evaluador.obtener_estadisticas_ejecucion()

        # Resumen final
        print(f"\n{'='*100}")
        print(f"✅ EVALUACION COMPLETA FINALIZADA EXITOSAMENTE")
        print(f"{'='*100}")

        if estadisticas:
            print(f"📁 Directorio de resultados: {estadisticas['directorio_ejecucion']}")
            print(f"🎯 Framework: {estadisticas['framework']}")
            print(f"📊 Tarea: {estadisticas['tarea_segmentacion']}")
            print(f"📸 Imágenes procesadas: {estadisticas['imagenes_procesadas']}")
            print(f"✅ Imágenes exitosas: {estadisticas['imagenes_exitosas']}")
            print(f"🤖 Modelos evaluados: {estadisticas['modelos_procesados']}")
            print(f"📄 Total archivos JSON: {estadisticas['total_archivos_json']}")

            if estadisticas['tiempo_total_procesamiento_ms'] > 0:
                tiempo_seg = estadisticas['tiempo_total_procesamiento_ms'] / 1000
                tiempo_min = tiempo_seg / 60
                print(f"⏱️  Tiempo total procesamiento: {tiempo_min:.1f} minutos ({tiempo_seg:.1f} segundos)")

            # Estadísticas de máscaras
            stats_mascaras = estadisticas.get('estadisticas_mascaras', {})
            if stats_mascaras and stats_mascaras.get('total_archivos', 0) > 0:
                print(f"\n💾 ESTADÍSTICAS DE MÁSCARAS:")
                print(f"   Total archivos NPZ: {stats_mascaras['total_archivos']}")
                print(f"   Tamaño total: {stats_mascaras['tamaño_total_mb']} MB")
                print(f"   Tamaño promedio: {stats_mascaras['tamaño_promedio_mb']} MB por archivo")
                print(f"   Rango: {stats_mascaras['tamaño_min_mb']} MB - {stats_mascaras['tamaño_max_mb']} MB")

        print(f"\n📂 RESUMEN POR MODELO:")
        for modelo, archivos in archivos_resultado.items():
            print(f"   {modelo}: {len(archivos)} evaluaciones completadas")
            if archivos:
                ejemplo_archivo = Path(archivos[0])
                print(f"      📍 Ubicación: {ejemplo_archivo.parent.name}/{ejemplo_archivo.name}")

        # Mostrar información sobre logs
        if estadisticas and estadisticas.get('estadisticas_logs'):
            logs_stats = estadisticas['estadisticas_logs']
            if logs_stats['total_archivos_log'] > 0:
                print(f"\n📝 LOGS GENERADOS:")
                print(f"   Archivos de log: {logs_stats['total_archivos_log']}")
                print(f"   Componentes activos: {logs_stats['componentes_activos']}")
                for detalle in logs_stats.get('archivos_detalle', [])[:5]:
                    print(f"      - {detalle['archivo']}: {detalle['tamaño_kb']:.2f} KB")
            else:
                print(f"\n📝 LOGS: Sin archivos generados (ejecución limpia)")

        print(f"\n✅ ANÁLISIS COMPLETADO CON ÉXITO")
        print(f"🎉 OneFormer Universal Segmentation - Evaluación Finalizada")

    except Exception as e:
        print(f"\n❌ ERROR CRITICO EN MAIN: {str(e)}")
        import traceback
        traceback.print_exc()
        raise

In [31]:
if __name__ == "__main__":
    main()


SISTEMA DE EVALUACION ONEFORMER (UNIVERSAL SEGMENTATION)

Tarea de segmentación: instance

Configuración de evaluación:
   Modelos: 4
      [0] oneformer-swin-large-coco (universal, COCO)
          Tareas: semantic, instance, panoptic
      [1] oneformer-swin-large-ade20k (universal, ADE20K)
          Tareas: semantic, instance, panoptic
      [2] oneformer-swin-tiny-ade20k (universal, ADE20K)
          Tareas: semantic, instance, panoptic
      [3] oneformer-dinat-large-coco (universal, COCO)
          Tareas: semantic, instance, panoptic
   Configuraciones de umbral: 4
      ultra_sensible: Detecta cambios mínimos - Máxima sensibilidad
      alta_sensibilidad: Sensibilidad alta para detección temprana
      sensibilidad_media: Balance entre precisión y recall
      baja_sensibilidad: Solo detecciones muy confiables
✅ Configuración validada correctamente

📁 Directorio de ejecución: /content/drive/MyDrive/TFM/oneformer/resultados/ejecucion_20251004_194437

RESUMEN DE CONFIGURACION:
  

preprocessor_config.json: 0.00B [00:00, ?B/s]

coco_panoptic.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/893M [00:00<?, ?B/s]

❌ Error en combinación 13


model.safetensors:   0%|          | 0.00/893M [00:00<?, ?B/s]


COMBINACION 14/16
Modelo: oneformer-dinat-large-coco | Config: alta_sensibilidad
Tarea: instance
❌ Error en combinación 14

COMBINACION 15/16
Modelo: oneformer-dinat-large-coco | Config: sensibilidad_media
Tarea: instance
❌ Error en combinación 15

COMBINACION 16/16
Modelo: oneformer-dinat-large-coco | Config: baja_sensibilidad
Tarea: instance
❌ Error en combinación 16

EVALUACION COMPLETA ONEFORMER FINALIZADA
Todos los resultados en: /content/drive/MyDrive/TFM/oneformer/resultados/ejecucion_20251004_194437
Análisis completo con librerías especializadas
Máscaras almacenadas en archivos NPZ separados

✅ EVALUACION COMPLETA FINALIZADA EXITOSAMENTE
📁 Directorio de resultados: /content/drive/MyDrive/TFM/oneformer/resultados/ejecucion_20251004_194437
🎯 Framework: OneFormer
📊 Tarea: instance
📸 Imágenes procesadas: 72
✅ Imágenes exitosas: 72
🤖 Modelos evaluados: 3
📄 Total archivos JSON: 12
⏱️  Tiempo total procesamiento: 15.3 minutos (916.2 segundos)

💾 ESTADÍSTICAS DE MÁSCARAS:
   Total arc